In [1]:
# from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2 ##STrAdaBoost.R2
# from TwoStageTrAdaBoostR2 import TwoStageTrAdaBoostR2 ##two-stage TrAdaBoost.R2

import pandas as pd
import sys
import numpy as np
from pandas import DataFrame
from sklearn.model_selection import train_test_split, KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns

from scipy.stats.stats import pearsonr
from math import sqrt
import statistics 
from scipy.stats import *
from scipy.spatial import distance

######### Instance Transfer repositories ####################
from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2
from sklearn.model_selection import GridSearchCV
from adapt.instance_based import KMM

print("Repositories uploaded!!")

Repositories uploaded!!


In [2]:
################################################################ Ailerons ###############################################
target_ailerons = ['goal']
colnames_ailerons = ['climbRate', 'Sgz', 'p', 'q', 'curPitch', 'curRoll', 'absRoll', 'diffClb', 'diffRollRate', 'diffDiffClb', 'SeTime1',
            'SeTime2', 'SeTime3', 'SeTime4', 'SeTime5', 'SeTime6', 'SeTime7', 'SeTime8', 'SeTime9', 'SeTime10', 'SeTime11', 'SeTime12', 
            'SeTime13', 'SeTime14', 'diffSeTime1', 'diffSeTime2', 'diffSeTime3', 'diffSeTime4', 'diffSeTime5', 'diffSeTime6', 'diffSeTime7',
            'diffSeTime8', 'diffSeTime9', 'diffSeTime10', 'diffSeTime11', 'diffSeTime12', 'diffSeTime13', 'diffSeTime14', 'alpha', 'Se', 'goal']
AileronsData_train_df = pd.read_csv('UCI_regression/Ailerons/ailerons.data', header = None, names = colnames_ailerons) 
print("Ailerons Data")
print("Training Set: ", AileronsData_train_df.shape)

AileronsData_test_df = pd.read_csv('UCI_regression/Ailerons/ailerons.test', header = None, names = colnames_ailerons) 
print("Testing Set: ", AileronsData_test_df.shape)

print("-------------------------------------------")

############################################### Standardization ###############################################
ailerons_cols = AileronsData_train_df.columns.difference(['goal'])
ss = StandardScaler()
AileronsData_train_df[ailerons_cols] = ss.fit_transform(AileronsData_train_df[ailerons_cols])
AileronsData_test_df[ailerons_cols] = ss.transform(AileronsData_test_df[ailerons_cols]) ## Use the same scale as the training data


#################### Splitting with small target set and large source and test set #############
AileronsData_source_df, AileronsData_tgt_df = train_test_split(AileronsData_train_df, test_size = 0.05) ## test_size = tgt size
# print(AileronsData_df_tgt.shape, AileronsData_df_source.shape, AileronsData_df_test.shape)

AileronsData_train_df = AileronsData_train_df.reset_index(drop = True)
AileronsData_tgt_df = AileronsData_tgt_df.reset_index(drop = True)
AileronsData_source_df = AileronsData_source_df.reset_index(drop = True)
print("Target Set: ", AileronsData_tgt_df.shape)
print("Source Set: ", AileronsData_source_df.shape)
print("Test Set: ", AileronsData_test_df.shape)


# ################################# Splitting into predictors and target data #################################
# AileronsData_df_test_y = AileronsData_test_df[target_ailerons]
# AileronsData_df_test_X = AileronsData_test_df.drop(target_ailerons, axis = 1)

# AileronsData_df_tgt_y = AileronsData_tgt_df[target_ailerons]
# AileronsData_df_tgt_X = AileronsData_tgt_df.drop(target_ailerons, axis = 1)

# AileronsData_df_source_y = AileronsData_source_df[target_ailerons]
# AileronsData_df_source_X = AileronsData_source_df.drop(target_ailerons, axis = 1)


# ############## Merging the datasets #################
# ailerons_X_df = pd.concat([AileronsData_df_source_X, AileronsData_df_tgt_X], ignore_index=True)
# ailerons_y_df = pd.concat([AileronsData_df_source_y, AileronsData_df_tgt_y], ignore_index=True)

# ailerons_np_train_X = ailerons_X_df.to_numpy()
# ailerons_np_train_y = ailerons_y_df.to_numpy()

# ailerons_np_test_X = AileronsData_df_test_X.to_numpy()
# ailerons_np_test_y = AileronsData_df_test_y.to_numpy()

# ailerons_np_train_y_list = ailerons_np_train_y.ravel()
# ailerons_np_test_y_list = ailerons_np_test_y.ravel()

# src_size_ailerons = len(AileronsData_df_source_y)
# tgt_size_ailerons = len(AileronsData_df_tgt_y)

# print("-------------------------------------------")
# ##############################################################################################################################################

Ailerons Data
Training Set:  (7154, 41)
Testing Set:  (6596, 41)
-------------------------------------------
Target Set:  (358, 41)
Source Set:  (6796, 41)
Test Set:  (6596, 41)


In [3]:
######################################## Variance Sampling: Phase 1###################################################
kmeans = KMeans(n_clusters = 15, random_state=0).fit(AileronsData_tgt_df)

AileronsData_alternate_df = AileronsData_tgt_df.copy()
AileronsData_alternate_df_np = AileronsData_tgt_df.to_numpy()

idxlist = []
AileronsData_new_df_list = []

for rowkm in kmeans.cluster_centers_:
    mindist = -99
    rowidx = 0
    idx = 0
    for row in AileronsData_alternate_df_np:
        dst = distance.euclidean(row, rowkm)

        if(dst >= mindist):
            mindist = dst
            rowidx = idx
            rowval = row

        idx = idx + 1

#     print("Row selected: ", rowidx) #Alternate_df.loc[rowidx,:]\
#     print("Min. distance: ", mindist)
#     print("Matrix shape: ", AileronsData_alternate_df_np.shape)
    AileronsData_new_df_list.append(rowval)
    AileronsData_alternate_df = np.delete(AileronsData_alternate_df_np, rowidx, 0)
    idxlist.append(rowidx)


AileronsData_new_df = pd.DataFrame(np.vstack(AileronsData_new_df_list))
AileronsData_new_df.shape

(15, 41)

In [4]:
##################################################### Variance Sampling: Phase 2 ################################################

AileronsData_alternate_source_df = AileronsData_source_df.copy()
AileronsData_alternate_source_df_np = AileronsData_alternate_source_df.to_numpy()

idxlist2 = []
AileronsData_final_df_list = []

for row_nm in AileronsData_new_df_list:
    min_dist = -99
    row_idx = 0
    idx_val = 0
    for row_alt in AileronsData_alternate_df_np:
        dst = distance.euclidean(row_alt, row_nm)
        if(dst >= mindist):
            min_dist = dst
            row_idx = idx_val
            row_val = row_alt

        idx_val = idx_val + 1

#     print("Row selected: ", row_idx) #Alternate_df.loc[rowidx,:]\
#     print("Min. distance: ", min_dist)
#     print("Matrix shape: ", AileronsData_alternate_source_df_np.shape)
    AileronsData_final_df_list.append(row_val)
    AileronsData_alternate_source_df_np = np.delete(AileronsData_alternate_source_df_np, row_idx, 0)
    idxlist2.append(row_idx)


AileronsData_final_df = pd.DataFrame(np.vstack(AileronsData_final_df_list), columns= AileronsData_alternate_source_df.columns)


print("Shape of source dataset before Variance Sampling: ",AileronsData_source_df.shape)
AileronsData_source_df = pd.DataFrame(np.vstack(AileronsData_alternate_source_df_np), columns= AileronsData_source_df.columns)
print("Shape of source dataset after Variance Sampling: ", AileronsData_source_df.shape)

print("----------------------------------------------")

print("Shape of target before Variance Sampling: ", AileronsData_tgt_df.shape)
AileronsData_tgt_df = pd.concat([AileronsData_tgt_df, AileronsData_final_df], ignore_index=True)
print("Shape of target after Variance Sampling: ", AileronsData_tgt_df.shape)

print("----------------------------------------------")

Shape of source dataset before Variance Sampling:  (6796, 41)
Shape of source dataset after Variance Sampling:  (6781, 41)
----------------------------------------------
Shape of target before Variance Sampling:  (358, 41)
Shape of target after Variance Sampling:  (373, 41)
----------------------------------------------


In [5]:
################################## Importance Sampling: Manhattan Distance ######################################################

AileronsData_source_df["ManDis"] = ""

AileronsData_tgt_df_mean = []
prow = AileronsData_tgt_df.mean()
AileronsData_tgt_df_mean = [prow.climbRate, prow.Sgz, prow.p, prow.q, prow.curPitch, prow.curRoll, prow.absRoll,
       prow.diffClb, prow.diffRollRate, prow.diffDiffClb, prow.SeTime1, prow.SeTime2, prow.SeTime3, prow.SeTime4, 
        prow.SeTime5, prow.SeTime6, prow.SeTime7, prow.SeTime8, prow.SeTime9, prow.SeTime10, prow.SeTime11, prow.SeTime12, 
        prow.SeTime13, prow.SeTime14, prow.diffSeTime1, prow.diffSeTime2, prow.diffSeTime3, prow.diffSeTime4, prow.diffSeTime5, 
        prow.diffSeTime6, prow.diffSeTime7, prow.diffSeTime8, prow.diffSeTime9, prow.diffSeTime10, prow.diffSeTime11, prow.diffSeTime12,
       prow.diffSeTime13, prow.diffSeTime14, prow.alpha, prow.Se, prow.goal]

rowidx = 0

for row in AileronsData_source_df.itertuples():
    row_list =[row.climbRate, row.Sgz, row.p, row.q, row.curPitch, row.curRoll, row.absRoll,
       row.diffClb, row.diffRollRate, row.diffDiffClb, row.SeTime1, row.SeTime2, row.SeTime3, row.SeTime4, 
        row.SeTime5, row.SeTime6, row.SeTime7, row.SeTime8, row.SeTime9, row.SeTime10, row.SeTime11, row.SeTime12, 
        row.SeTime13, row.SeTime14, row.diffSeTime1, row.diffSeTime2, row.diffSeTime3, row.diffSeTime4, row.diffSeTime5, 
        row.diffSeTime6, row.diffSeTime7, row.diffSeTime8, row.diffSeTime9, row.diffSeTime10, row.diffSeTime11, row.diffSeTime12,
       row.diffSeTime13, row.diffSeTime14, row.alpha, row.Se, row.goal]

    man_dis = 0
    for i in range(0, len(row_list)):
        tempval = AileronsData_tgt_df_mean[i] - row_list[i]
        man_dis = man_dis + abs(tempval)

    AileronsData_source_df.loc[rowidx,"ManDis"] = man_dis
    rowidx = rowidx + 1

AileronsData_source_df = AileronsData_source_df.sort_values(by =['ManDis'])
AileronsData_source_df = AileronsData_source_df.head(1500) 
AileronsData_source_df = AileronsData_source_df.drop(['ManDis'], axis =1)
AileronsData_source_df = AileronsData_source_df.reset_index(drop=True)

# print("Target Set: ",AileronsData_tgt_df.shape)
# print("Source Set: ",AileronsData_source_df.shape)
# print("Test Set: ",AileronsData_test_df.shape)


################################# Splitting into predictors and target data #################################
print("Target Set: ", AileronsData_tgt_df.shape)
print("Source Set: ", AileronsData_source_df.shape)
print("Test Set: ", AileronsData_test_df.shape)


AileronsData_df_test_y = AileronsData_test_df[target_ailerons]
AileronsData_df_test_X = AileronsData_test_df.drop(target_ailerons, axis = 1)

AileronsData_df_tgt_y = AileronsData_tgt_df[target_ailerons]
AileronsData_df_tgt_X = AileronsData_tgt_df.drop(target_ailerons, axis = 1)

AileronsData_df_source_y = AileronsData_source_df[target_ailerons]
AileronsData_df_source_X = AileronsData_source_df.drop(target_ailerons, axis = 1)


############## Merging the datasets #################
ailerons_X_df = pd.concat([AileronsData_df_tgt_X, AileronsData_df_source_X], ignore_index=True)
ailerons_y_df = pd.concat([AileronsData_df_tgt_y, AileronsData_df_source_y], ignore_index=True)

ailerons_np_train_X = ailerons_X_df.to_numpy()
ailerons_np_train_y = ailerons_y_df.to_numpy()

ailerons_np_test_X = AileronsData_df_test_X.to_numpy()
ailerons_np_test_y = AileronsData_df_test_y.to_numpy()

ailerons_np_train_y_list = ailerons_np_train_y.ravel()
ailerons_np_test_y_list = ailerons_np_test_y.ravel()

src_size_ailerons = len(AileronsData_df_source_y)
tgt_size_ailerons = len(AileronsData_df_tgt_y)

print("-------------------------------------------")

###############################################################################################################################################

Target Set:  (373, 41)
Source Set:  (1500, 41)
Test Set:  (6596, 41)
-------------------------------------------


In [10]:
#################################### STrAdaBoost.R2 Active Sampling Ailerons ################################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

sample_size = [len(AileronsData_df_tgt_X), len(AileronsData_df_source_X)]
n_estimators = 100
steps = 30
fold = 10
random_state = np.random.RandomState(1)

r2scorelist_stradaboost_ailerons = []
rmselist_stradaboost_ailerons = []

for x in range(0, 20):

    model_stradaboost_ailerons = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth=6),
                        n_estimators = n_estimators, sample_size = sample_size,
                        steps = steps, fold = fold, random_state = random_state)


    model_stradaboost_ailerons.fit(ailerons_np_train_X, ailerons_np_train_y_list)


    y_pred_stradaboost_ailerons = model_stradaboost_ailerons.predict(ailerons_np_test_X)

    mse_stradaboost_ailerons = sqrt(mean_squared_error(ailerons_np_test_y, y_pred_stradaboost_ailerons))
    rmselist_stradaboost_ailerons.append(mse_stradaboost_ailerons)
        
    r2_score_stradaboost_ailerons = pearsonr(ailerons_np_test_y_list, y_pred_stradaboost_ailerons)
    r2_score_stradaboost_ailerons = (r2_score_stradaboost_ailerons[0])**2
    r2scorelist_stradaboost_ailerons.append(r2_score_stradaboost_ailerons)

print("RMSE of STrAdaboostR2:", rmselist_stradaboost_ailerons)
print("R^2 of STrAdaboostR2:", r2scorelist_stradaboost_ailerons)

print("\n")

print("RMSE of STrAdaboostR2:", statistics.mean(rmselist_stradaboost_ailerons), statistics.stdev(rmselist_stradaboost_ailerons))
print("R^2 of STrAdaboostR2:", statistics.mean(r2scorelist_stradaboost_ailerons), statistics.stdev(r2scorelist_stradaboost_ailerons))

print("-------------------------------------------")

Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
RMSE of STrAdaboostR2: [0.00018653840674816383, 0.000189810904166031, 0.0001873658474340575, 0.00019085531291090202, 0.00019114973737215766, 0.00019043394736071044, 0.0001888067344590376, 0.00018804314636032746, 0.00018824536253958136, 0.00

In [8]:
########################### Two-Stage TrAdaBoost.R2 Ailerons #######################################

src_idx_ailerons = np.arange(start = 0, stop = (src_size_ailerons - 1), step = 1)
tgt_idx_ailerons = np.arange(start = src_size_ailerons, stop = ((src_size_ailerons + tgt_size_ailerons)-1), step = 1)

def get_estimator(**kwargs):
    return DecisionTreeRegressor(max_depth = 6)

kwargs_TwoTrAda = {'steps': 30,
                    'fold': 10,
                  'learning_rate': 0.1}


from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2

print("Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

r2scorelist_TwoTrAda_ailerons = []
rmselist_TwoTrAda_ailerons = []

for x in range(0, 20):

    model_TwoTrAda_ailerons = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 100, cv = 10) 
    model_TwoTrAda_ailerons.fit(ailerons_np_train_X, ailerons_np_train_y_list, src_idx_ailerons, tgt_idx_ailerons)

    y_pred_TwoTrAda_ailerons = model_TwoTrAda_ailerons.predict(ailerons_np_test_X)

    mse_TwoTrAda_ailerons = sqrt(mean_squared_error(ailerons_np_test_y, y_pred_TwoTrAda_ailerons))
    rmselist_TwoTrAda_ailerons.append(mse_TwoTrAda_ailerons)
        
    r2_score_TwoTrAda_ailerons = pearsonr(ailerons_np_test_y_list, y_pred_TwoTrAda_ailerons)
    r2_score_TwoTrAda_ailerons = (r2_score_TwoTrAda_ailerons[0])**2
    r2scorelist_TwoTrAda_ailerons.append(r2_score_TwoTrAda_ailerons)


print("RMSE of TrAdaboost.R2:", statistics.mean(rmselist_TwoTrAda_ailerons))
print("R^2 of TrAdaboost.R2:", statistics.mean(r2scorelist_TwoTrAda_ailerons))
print("\n")
print("RMSE of TrAdaboost.R2:", rmselist_TwoTrAda_ailerons)
print("R^2 of TrAdaboostR2:", r2scorelist_TwoTrAda_ailerons)


print("-------------------------------------------")

Two-Stage TrAdaboost.R2
-------------------------------------------
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set

In [6]:
########################### AdaBoostR2 Transfer Learning Ailerons #####################################################
from sklearn.ensemble import AdaBoostRegressor

print("Adaboost.R2 Transfer Learning")
print("-------------------------------------------")

r2scorelist_AdaTL_ailerons = []
rmselist_AdaTL_ailerons = []

for x in range(0, 30):

    model_AdaTL_ailerons = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 6), learning_rate = 0.1, n_estimators = 100) 
    model_AdaTL_ailerons.fit(ailerons_np_train_X, ailerons_np_train_y_list)

    y_pred_AdaTL_ailerons = model_AdaTL_ailerons.predict(ailerons_np_test_X)

    mse_AdaTL_ailerons = sqrt(mean_squared_error(ailerons_np_test_y, y_pred_AdaTL_ailerons))
    rmselist_AdaTL_ailerons.append(mse_AdaTL_ailerons)
        
    r2_score_AdaTL_ailerons = pearsonr(ailerons_np_test_y_list, y_pred_AdaTL_ailerons)
    r2_score_AdaTL_ailerons = (r2_score_AdaTL_ailerons[0])**2
    r2scorelist_AdaTL_ailerons.append(r2_score_AdaTL_ailerons)

print("RMSE of Adaboost.R2:", statistics.mean(rmselist_AdaTL_ailerons))
print("R^2 of Adaboost.R2:", statistics.mean(r2scorelist_AdaTL_ailerons))
print("\n")
print("RMSE of Adaboost.R2:", rmselist_AdaTL_ailerons)
print("R^2 of Adaboost.R2:", r2scorelist_AdaTL_ailerons)


print("-------------------------------------------")

Adaboost.R2 Transfer Learning
-------------------------------------------
RMSE of Adaboost.R2: 0.0001951552208269592
R^2 of Adaboost.R2: 0.7726247596150608


RMSE of Adaboost.R2: [0.00019595313556389278, 0.00019407133149892483, 0.00019601181825972422, 0.0001954022754515077, 0.00019519023336717658, 0.00019474587162315903, 0.0001950705768073821, 0.00019608253719637705, 0.00019476537149207756, 0.0001952450050474583, 0.00019779674594388433, 0.00019426071376450418, 0.000193893994537166, 0.00019509715787169402, 0.00019468481978249396, 0.00019583859105140586, 0.0001960477238821678, 0.0001959256089267998, 0.0001943649510498122, 0.000194538431758311, 0.0001951850160876205, 0.00019523905455156806, 0.00019566656820353307, 0.0001949762679623272, 0.0001934223429704112, 0.00019456146239891744, 0.00019538987138243463, 0.00019433892789750423, 0.00019502694244499195, 0.00019586327603354843]
R^2 of Adaboost.R2: [0.7729344508608056, 0.7765145247649465, 0.7665704014531523, 0.7713984417354421, 0.7715578518

In [7]:
######################### Gradient Boosting Regression Transfer Learning Ailerons #######################################
from sklearn.ensemble import GradientBoostingRegressor


print("Gradient Boosting Regression Transfer Learning")
print("-------------------------------------------")

r2scorelist_GBRTL_ailerons = []
rmselist_GBRTL_ailerons = []

for x in range(0, 20):

    model_GBRTL_ailerons = GradientBoostingRegressor(learning_rate = 0.1, max_depth = 6, n_estimators = 100)
    model_GBRTL_ailerons.fit(ailerons_np_train_X, ailerons_np_train_y_list)

    y_pred_GBRTL_ailerons = model_GBRTL_ailerons.predict(AileronsData_df_test_X) ##Using dataframe instead of the numpy matrix

    mse_GBRTL_ailerons = sqrt(mean_squared_error(ailerons_np_test_y, y_pred_GBRTL_ailerons))
    rmselist_GBRTL_ailerons.append(mse_GBRTL_ailerons)
        
    r2_score_GBRTL_ailerons = pearsonr(ailerons_np_test_y_list, y_pred_GBRTL_ailerons)
    r2_score_GBRTL_ailerons = (r2_score_GBRTL_ailerons[0])**2
    r2scorelist_GBRTL_ailerons.append(r2_score_GBRTL_ailerons)

    
print("RMSE of GBRTL:", statistics.mean(rmselist_GBRTL_ailerons))
print("R^2 of GBRTL:", statistics.mean(r2scorelist_GBRTL_ailerons))
print("\n")
print("RMSE of GBRTL:", rmselist_GBRTL_ailerons)
print("R^2 of GBRTL:", r2scorelist_GBRTL_ailerons)


print("-------------------------------------------")

Gradient Boosting Regression Transfer Learning
-------------------------------------------
RMSE of GBRTL: 0.0001895369735514958
R^2 of GBRTL: 0.7830693129081961


RMSE of GBRTL: [0.00019078064490331108, 0.00019065142948389952, 0.00019079018165038723, 0.00018761972603949454, 0.0001898670206287818, 0.00019049776873561613, 0.000188244888272926, 0.00019153582833195644, 0.00018866618543196816, 0.00018949132399937338, 0.00018629023845475717, 0.0001906718215463847, 0.0001895218338079132, 0.00018942979652755316, 0.000188079053424787, 0.00018963174968158526, 0.00019056196175690893, 0.00018876960559616586, 0.0001898078848750164, 0.00018983052788112994]
R^2 of GBRTL: [0.7803413288252402, 0.7808517664613813, 0.7805260033480872, 0.787306783988316, 0.7822259543949277, 0.7812709368481537, 0.7855524392714588, 0.7790800115878448, 0.7846341421197356, 0.7831483658696062, 0.7893295436846223, 0.7806161090091442, 0.7833359558906222, 0.7830286320806953, 0.7858821941383876, 0.783042480651071, 0.78113582950098

In [9]:
####################################### Elevators #########################################################
target_elevators = ['Goal']
colnames_elevators = ['climbRate', 'Sgz', 'p', 'q', 'curRoll', 'absRoll', 'diffClb', 'diffRollRate', 'diffDiffClb', 'SaTime1', 'SaTime2', 
                      'SaTime3', 'SaTime4', 'diffSaTime1', 'diffSaTime2', 'diffSaTime3', 'diffSaTime4', 'Sa', 'Goal']
Elevators_train_df = pd.read_csv('UCI_regression/Elevators/elevators.data', header = None, names = colnames_elevators)
print("Elevators Data")
print(Elevators_train_df.shape)

Elevators_test_df = pd.read_csv('UCI_regression/Elevators/elevators.test', header = None, names = colnames_elevators)
print(Elevators_test_df.shape)

############################################### Standardization ###############################################
elevators_cols = Elevators_train_df.columns.difference(['Goal'])
ss = StandardScaler()
Elevators_train_df[elevators_cols] = ss.fit_transform(Elevators_train_df[elevators_cols])
Elevators_test_df[elevators_cols] = ss.transform(Elevators_test_df[elevators_cols]) ## Use the same scale as the training data

#################### Splitting with small target set and large source and test set #############
Elevators_source_df, Elevators_tgt_df = train_test_split(Elevators_train_df, test_size = 0.05) ## test_size = tgt size
# print(Elevators_df_tgt.shape, Elevators_df_source.shape, Elevators_df_test.shape)

Elevators_tgt_df = Elevators_tgt_df.reset_index(drop = True)
Elevators_source_df = Elevators_source_df.reset_index(drop = True)
print("Target Set: ", Elevators_tgt_df.shape)
print("Source Set: ", Elevators_source_df.shape)
print("Test Set: ", Elevators_test_df.shape)

################################# Splitting into predictors and target data #################################
# Elevators_test_df_y = Elevators_test_df[target_elevators]
# Elevators_test_df_X = Elevators_test_df.drop(target_elevators, axis = 1)

# Elevators_tgt_df_y = Elevators_tgt_df[target_elevators]
# Elevators_tgt_df_X = Elevators_tgt_df.drop(target_elevators, axis = 1)

# Elevators_source_df_y = Elevators_source_df[target_elevators]
# Elevators_source_df_X = Elevators_source_df.drop(target_elevators, axis = 1)


# ############## Merging the datasets #################
# elevators_X_df = pd.concat([Elevators_source_df_X, Elevators_tgt_df_X], ignore_index=True)
# elevators_y_df = pd.concat([Elevators_source_df_y, Elevators_tgt_df_y], ignore_index=True)

# elevators_np_train_X = elevators_X_df.to_numpy()
# elevators_np_train_y = elevators_y_df.to_numpy()

# elevators_np_test_X = Elevators_test_df_X.to_numpy()
# elevators_np_test_y = Elevators_test_df_y.to_numpy()

# elevators_np_train_y_list = elevators_np_train_y.ravel()
# elevators_np_test_y_list = elevators_np_test_y.ravel()

# src_size_elevators = len(Elevators_source_df_y)
# tgt_size_elevators = len(Elevators_tgt_df_y)

# print("---------------------------")

# ###############################################################################################################################################

Elevators Data
(8752, 19)
(7847, 19)
Target Set:  (438, 19)
Source Set:  (8314, 19)
Test Set:  (7847, 19)


In [10]:
######################################## Variance Sampling: Phase 1 ###################################################
kmeans = KMeans(n_clusters = 20, random_state=0).fit(Elevators_tgt_df)

Elevators_alternate_df = Elevators_tgt_df.copy()
Elevators_alternate_df_np = Elevators_tgt_df.to_numpy()

idxlist = []
Elevators_new_df_list = []

for rowkm in kmeans.cluster_centers_:
    mindist = -99
    rowidx = 0
    idx = 0
    for row in Elevators_alternate_df_np:
        dst = distance.euclidean(row, rowkm)

        if(dst >= mindist):
            mindist = dst
            rowidx = idx
            rowval = row

        idx = idx + 1

#     print("Row selected: ", rowidx) #Alternate_df.loc[rowidx,:]\
#     print("Min. distance: ", mindist)
#     print("Matrix shape: ", Elevators_alternate_df_np.shape)
    Elevators_new_df_list.append(rowval)
    Elevators_alternate_df = np.delete(Elevators_alternate_df_np, rowidx, 0)
    idxlist.append(rowidx)


Elevators_new_df = pd.DataFrame(np.vstack(Elevators_new_df_list))
Elevators_new_df.shape

##################################################### Variance Sampling: Phase 2 ################################################

Elevators_alternate_source_df = Elevators_source_df.copy()
Elevators_alternate_source_df_np = Elevators_alternate_source_df.to_numpy()

idxlist2 = []
Elevators_final_df_list = []

for row_nm in Elevators_new_df_list:
    min_dist = -99
    row_idx = 0
    idx_val = 0
    for row_alt in Elevators_alternate_df_np:
        dst = distance.euclidean(row_alt, row_nm)
        if(dst >= mindist):
            min_dist = dst
            row_idx = idx_val
            row_val = row_alt

        idx_val = idx_val + 1

#     print("Row selected: ", row_idx) #Alternate_df.loc[rowidx,:]\
#     print("Min. distance: ", min_dist)
#     print("Matrix shape: ", Elevators_alternate_source_df_np.shape)
    Elevators_final_df_list.append(row_val)
    Elevators_alternate_source_df_np = np.delete(Elevators_alternate_source_df_np, row_idx, 0)
    idxlist2.append(row_idx)


Elevators_final_df = pd.DataFrame(np.vstack(Elevators_final_df_list), columns= Elevators_alternate_source_df.columns)

print("Shape of source before :",Elevators_source_df.shape)
Elevators_source_df = pd.DataFrame(np.vstack(Elevators_alternate_source_df_np), columns= Elevators_source_df.columns)
print("Shape of source after :", Elevators_source_df.shape)

print("----------------------------------------------")
print("Shape of target before :", Elevators_tgt_df.shape)
Elevators_tgt_df = pd.concat([Elevators_tgt_df, Elevators_final_df], ignore_index=True)
print("Shape of target after :", Elevators_tgt_df.shape)

print("----------------------------------------------")

Shape of source before : (8314, 19)
Shape of source after : (8294, 19)
----------------------------------------------
Shape of target before : (438, 19)
Shape of target after : (458, 19)
----------------------------------------------


In [11]:
################################## Importance Sampling: Manhattan Distance ######################################################

Elevators_source_df["ManDis"] = ""

Elevators_tgt_df_mean = []
prow = Elevators_tgt_df.mean()
Elevators_tgt_df_mean = [prow.climbRate, prow.Sgz, prow.p, prow.q, prow.curRoll, prow.absRoll,
       prow.diffClb, prow.diffRollRate, prow.diffDiffClb, prow.SaTime1, prow.SaTime2, prow.SaTime3, prow.SaTime4, 
        prow.diffSaTime1, prow.diffSaTime2, prow.diffSaTime3, prow.diffSaTime4, prow.Sa, prow.Goal]

rowidx = 0

for row in Elevators_source_df.itertuples():
    row_list =[row.climbRate, row.Sgz, row.p, row.q, row.curRoll, row.absRoll, row.diffClb, 
            row.diffRollRate, row.diffDiffClb, row.SaTime1, row.SaTime2, row.SaTime3, row.SaTime4, 
               row.diffSaTime1, row.diffSaTime2, row.diffSaTime3, row.diffSaTime4, row.Sa, row.Goal]

    man_dis = 0
    for i in range(0, len(row_list)):
        tempval = Elevators_tgt_df_mean[i] - row_list[i]
        man_dis = man_dis + abs(tempval)

    Elevators_source_df.loc[rowidx,"ManDis"] = man_dis
    rowidx = rowidx + 1

Elevators_source_df = Elevators_source_df.sort_values(by =['ManDis'])
Elevators_tgt_source_df = Elevators_source_df.head(2078) ## 600 instances chosen for Kinematics Dataset
Elevators_source_df = Elevators_source_df.iloc[2078:]

Elevators_source_df = Elevators_source_df.drop(['ManDis'], axis =1)
Elevators_tgt_source_df = Elevators_tgt_source_df.drop(['ManDis'], axis =1)

Elevators_tgt_df = pd.concat([Elevators_tgt_df, Elevators_tgt_source_df], ignore_index=True) ### This line is used only for STrAdaBoost.R2 and not for TrAdaBoost.R2

Elevators_tgt_df = Elevators_tgt_df.reset_index(drop=True)
Elevators_source_df = Elevators_source_df.reset_index(drop=True)


#################### Splitting with small target set and large source and test set #############
print("Target Set: ", Elevators_tgt_df.shape)
print("Source Set: ", Elevators_source_df.shape)
print("Test Set: ", Elevators_test_df.shape)


Elevators_test_df_y = Elevators_test_df[target_elevators]
Elevators_test_df_X = Elevators_test_df.drop(target_elevators, axis = 1)

Elevators_tgt_df_y = Elevators_tgt_df[target_elevators]
Elevators_tgt_df_X = Elevators_tgt_df.drop(target_elevators, axis = 1)

Elevators_source_df_y = Elevators_source_df[target_elevators]
Elevators_source_df_X = Elevators_source_df.drop(target_elevators, axis = 1)


############## Merging the datasets #################
elevators_X_df = pd.concat([Elevators_tgt_df_X, Elevators_source_df_X], ignore_index=True)
elevators_y_df = pd.concat([Elevators_tgt_df_y, Elevators_source_df_y], ignore_index=True)

elevators_np_train_X = elevators_X_df.to_numpy()
elevators_np_train_y = elevators_y_df.to_numpy()

elevators_np_test_X = Elevators_tgt_df_X.to_numpy()
elevators_np_test_y = Elevators_tgt_df_y.to_numpy()

elevators_np_train_y_list = elevators_np_train_y.ravel()
elevators_np_test_y_list = elevators_np_test_y.ravel()

src_size_elevators = len(Elevators_source_df_y)
tgt_size_elevators = len(Elevators_tgt_df_y)

print("---------------------------")

###############################################################################################################################################

Target Set:  (2536, 19)
Source Set:  (6216, 19)
Test Set:  (7847, 19)
---------------------------


In [8]:
#################################### STrAdaBoost.R2 Active Sampling Elevators ################################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

sample_size = [len(Elevators_tgt_df_X), len(Elevators_source_df_X)]
n_estimators = 100
steps = 30
fold = 10
random_state = np.random.RandomState(1)

r2scorelist_stradaboost_elevators = []
rmselist_stradaboost_elevators = []

for x in range(0, 20):

    model_stradaboost_elevators = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth=6),
                        n_estimators = n_estimators, sample_size = sample_size,
                        steps = steps, fold = fold, random_state = random_state)


    model_stradaboost_elevators.fit(elevators_np_train_X, elevators_np_train_y_list)


    y_pred_stradaboost_elevators = model_stradaboost_elevators.predict(elevators_np_test_X)

    mse_stradaboost_elevators = sqrt(mean_squared_error(elevators_np_test_y, y_pred_stradaboost_elevators))
    rmselist_stradaboost_elevators.append(mse_stradaboost_elevators)
        
    r2_score_stradaboost_elevators = pearsonr(elevators_np_test_y_list, y_pred_stradaboost_elevators)
    r2_score_stradaboost_elevators = (r2_score_stradaboost_elevators[0])**2
    r2scorelist_stradaboost_elevators.append(r2_score_stradaboost_elevators)


print("RMSE of STrAdaboostR2:", statistics.mean(rmselist_stradaboost_elevators))
print("R^2 of STrAdaboostR2:", statistics.mean(r2scorelist_stradaboost_elevators))
print("\n")
print("RMSE of STrAdaboostR2:", rmselist_stradaboost_elevators)
print("R^2 of STrAdaboostR2:", r2scorelist_stradaboost_elevators)


print("-------------------------------------------")

Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
RMSE of STrAdaboostR2: 0.0030410629395618218
R^2 of STrAdaboostR2: 0.8194621213682728


RMSE of STrAdaboostR2: [0.002994308060777424, 0.0031586784175449143, 0.0030899128906734154, 0.0029370613376077503, 0.002979925497627271, 0.0032391763097

In [13]:
########################### TwoStagetrAdaBoostR2 Elevators #######################################
from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2

############# Transfer Learning specifications #########################
src_idx_elevators = np.arange(start=0, stop=(src_size_elevators - 1), step=1)
tgt_idx_elevators = np.arange(start=src_size_elevators, stop=((src_size_elevators + tgt_size_elevators)-1), step=1)

def get_estimator(**kwargs):
    return DecisionTreeRegressor(max_depth = 6)

kwargs_TwoTrAda = {'steps': 30,
                    'fold': 10,
                  'learning_rate': 0.1}



print("Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

r2scorelist_TwoTrAda_elevators = []
rmselist_TwoTrAda_elevators = []

for x in range(0, 20):

    model_TwoTrAda_elevators = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 100) 
    model_TwoTrAda_elevators.fit(elevators_np_train_X, elevators_np_train_y_list, src_idx_elevators, tgt_idx_elevators)

    y_pred_TwoTrAda_elevators = model_TwoTrAda_elevators.predict(elevators_np_test_X)

    mse_TwoTrAda_elevators = sqrt(mean_squared_error(elevators_np_test_y, y_pred_TwoTrAda_elevators))
    rmselist_TwoTrAda_elevators.append(mse_TwoTrAda_elevators)
        
    r2_score_TwoTrAda_elevators = pearsonr(elevators_np_test_y_list, y_pred_TwoTrAda_elevators)
    r2_score_TwoTrAda_elevators = (r2_score_TwoTrAda_elevators[0])**2

    r2scorelist_TwoTrAda_elevators.append(r2_score_TwoTrAda_elevators)


print("RMSE of TrAdaboost.R2:", statistics.mean(rmselist_TwoTrAda_elevators))
print("R^2 of TrAdaboostR2:", statistics.mean(r2scorelist_TwoTrAda_elevators))
print("\n")
print("RMSE of TrAdaboost.R2:", rmselist_TwoTrAda_elevators)
print("R^2 of TrAdaboostR2:", r2scorelist_TwoTrAda_elevators)


print("-------------------------------------------")

Two-Stage TrAdaboost.R2
-------------------------------------------
RMSE of TrAdaboost.R2: 0.004252382013001438
R^2 of TrAdaboostR2: 0.3832258356787882


RMSE of TrAdaboost.R2: [0.004275919983996502, 0.004065268426812282, 0.004113395961403267, 0.004212334391192323, 0.004288498406503932, 0.004378834846288086, 0.003614079335797574, 0.004065267981158002, 0.004249356249477287, 0.004538052419205431, 0.004073175603346029, 0.004466231064563387, 0.004515145948876253, 0.0041883074646359994, 0.004509266884755467, 0.004504083203488585, 0.004327980331242717, 0.004533669298007648, 0.004063489404373452, 0.00406528305490454]
R^2 of TrAdaboostR2: [0.37640847351511997, 0.43050902808961616, 0.420990773200773, 0.3959881188360485, 0.36358452529870366, 0.3539248754337524, 0.5509400271571356, 0.43050948863942673, 0.38802007067797284, 0.30097707678485297, 0.43503011081893417, 0.3238792243887958, 0.3137875854906619, 0.40114826252136027, 0.3100011831667205, 0.3145085660294332, 0.3650281067495453, 0.31839843090

In [12]:
######################### Gradient Boosting Regression Transfer Learning Elevators #######################################
from sklearn.ensemble import GradientBoostingRegressor


print("Gradient Boosting Regression Transfer Learning")
print("-------------------------------------------")

r2scorelist_GBRTL_elevators = []
rmselist_GBRTL_elevators = []

for x in range(0, 20):

    model_GBRTL_elevators = GradientBoostingRegressor(learning_rate = 0.1, max_depth = 6, n_estimators = 100)
    model_GBRTL_elevators.fit(elevators_np_train_X, elevators_np_train_y_list)

    y_pred_GBRTL_elevators = model_GBRTL_elevators.predict(elevators_np_test_X) ##Using dataframe instead of the numpy matrix

    mse_GBRTL_elevators = sqrt(mean_squared_error(elevators_np_test_y, y_pred_GBRTL_elevators))
    rmselist_GBRTL_elevators.append(mse_GBRTL_elevators)
        
    r2_score_GBRTL_elevators = pearsonr(elevators_np_test_y_list, y_pred_GBRTL_elevators)
    r2_score_GBRTL_elevators = (r2_score_GBRTL_elevators[0])**2
    r2scorelist_GBRTL_elevators.append(r2_score_GBRTL_elevators)


print("RMSE of GBRTL:", statistics.mean(rmselist_GBRTL_elevators))
print("R^2 of GBRTL:", statistics.mean(r2scorelist_GBRTL_elevators))
print("\n")
print("RMSE of GBRTL:", rmselist_GBRTL_elevators)
print("R^2 of GBRTL:", r2scorelist_GBRTL_elevators)


print("-------------------------------------------")

Gradient Boosting Regression Transfer Learning
-------------------------------------------
RMSE of GBRTL: 0.00161401693713203
R^2 of GBRTL: 0.912921907605285


RMSE of GBRTL: [0.00161401693713203, 0.00161401693713203, 0.00161401693713203, 0.00161401693713203, 0.0016140169371320302, 0.0016140169371320298, 0.00161401693713203, 0.00161401693713203, 0.0016140169371320298, 0.00161401693713203, 0.00161401693713203, 0.0016140169371320298, 0.00161401693713203, 0.0016140169371320298, 0.0016140169371320298, 0.00161401693713203, 0.00161401693713203, 0.00161401693713203, 0.00161401693713203, 0.0016140169371320298]
R^2 of GBRTL: [0.9129219076052852, 0.9129219076052852, 0.9129219076052848, 0.912921907605285, 0.9129219076052852, 0.9129219076052848, 0.9129219076052852, 0.912921907605285, 0.9129219076052852, 0.912921907605285, 0.912921907605285, 0.9129219076052852, 0.912921907605285, 0.912921907605285, 0.9129219076052848, 0.9129219076052852, 0.9129219076052848, 0.912921907605285, 0.912921907605285, 0.9

In [20]:
################################### Abalone ###########################################
#### range: [0.0 - 1.130]
#### Mid of correlation variable: Whole_weight
#### [0, 0.12] [0.12, 0.15], [0.15, 1.130]
#### Target variable: Rings
#######################################################################################
target_var_abalone = ['Rings']
colnames_abalone = ['Sex', 'Length', 'Diameter', 'Height', 'Whole_weight', 'Shucked_weight', 'Viscera_weight', 'Shell_weight', 'Rings']
AbaloneData_df = pd.read_csv('UCI_regression/Abalone/abalone.data', header = None, names = colnames_abalone)

gender = {'M': 1,'F': 2, 'I': 3} 
AbaloneData_df.Sex = [gender[item] for item in AbaloneData_df.Sex] 

print("Abalone Data")
print(AbaloneData_df.shape)

########## Corr Abalone ################
# print("The correlation matrix is: ")
# print(AbaloneData_df.corr()['Rings'])

##################### Splitting in 3 equal parts #######################################
# print(AbaloneData_df.sort_values('Whole_weight')['Whole_weight'])
# print(len(AbaloneData_df.loc[(AbaloneData_df['Whole_weight'] <= 0.5)]))
# print(len(AbaloneData_df.loc[(AbaloneData_df['Whole_weight'] > 0.5) & (AbaloneData_df['Whole_weight'] <= 1.0)]))
# print(len(AbaloneData_df.loc[(AbaloneData_df['Whole_weight'] > 1.0)]))

#################### Splitting with small target set and large source and test set #############
# AbaloneData_df_remain, AbaloneData_df_tgt = train_test_split(AbaloneData_df, test_size = 0.05) ## test_size = tgt size
# AbaloneData_df_source, AbaloneData_df_test = train_test_split(AbaloneData_df_remain, test_size = 0.3) ## test_size = tgt size
# print(AbaloneData_df_tgt.shape, AbaloneData_df_source.shape, AbaloneData_df_test.shape)


drop_col_abalone = ['Whole_weight']

abalone_tgt_df = AbaloneData_df.loc[(AbaloneData_df['Whole_weight'] <= 0.5)]
abalone_tgt_df = abalone_tgt_df.drop(drop_col_abalone, axis = 1)
abalone_tgt_df = abalone_tgt_df.reset_index(drop=True)
print("Target Set: ",abalone_tgt_df.shape)

abalone_source1_df = AbaloneData_df.loc[(AbaloneData_df['Whole_weight'] > 0.5) & (AbaloneData_df['Whole_weight'] <= 1.0)]
abalone_source1_df = abalone_source1_df.drop(drop_col_abalone, axis = 1)
abalone_source1_df = abalone_source1_df.reset_index(drop=True)
print("Source Set 1: ",abalone_source1_df.shape)

abalone_source2_df = AbaloneData_df.loc[(AbaloneData_df['Whole_weight'] > 1.0)]
abalone_source2_df = abalone_source2_df.drop(drop_col_abalone, axis = 1)
abalone_source2_df = abalone_source2_df.reset_index(drop=True)
print("Source Set 2: ",abalone_source2_df.shape)


################################# Standardization #################################
from sklearn.preprocessing import StandardScaler
abalone_cols = abalone_tgt_df.columns.difference(['Rings'])

ss = StandardScaler()
abalone_tgt_df[abalone_cols] = ss.fit_transform(abalone_tgt_df[abalone_cols])
abalone_source1_df[abalone_cols] = ss.fit_transform(abalone_source1_df[abalone_cols])
abalone_source2_df[abalone_cols] = ss.fit_transform(abalone_source2_df[abalone_cols])


################################# Concatenating the source datasets #################################
abalone_source_df = pd.concat([abalone_source1_df, abalone_source2_df], ignore_index = True)
abalone_source_df = abalone_source_df.reset_index(drop = True)
print("Final Source Set: ",abalone_source_df.shape)


# #################### Splitting into features and target ####################
# target_abalone = ['Rings']

# abalone_source_df_y = abalone_source_df[target_abalone]
# abalone_source_df_X = abalone_source_df.drop(target_abalone, axis = 1)

# features_abalone = abalone_source_df_X.columns
# print("---------------------------")
# ##############################################################################################################################################

Abalone Data
(4177, 9)
Target Set:  (1241, 8)
Source Set 1:  (1465, 8)
Source Set 2:  (1471, 8)
Final Source Set:  (2936, 8)


In [21]:
######################################## Variance Sampling: Phase 1 ###################################################
kmeans = KMeans(n_clusters = 60, random_state=0).fit(abalone_tgt_df)

abalone_alternate_df = abalone_tgt_df.copy()
abalone_alternate_df_np = abalone_tgt_df.to_numpy()

idxlist = []
abalone_new_df_list = []

for rowkm in kmeans.cluster_centers_:
    mindist = -99
    rowidx = 0
    idx = 0
    for row in abalone_alternate_df_np:
        dst = distance.euclidean(row, rowkm)

        if(dst >= mindist):
            mindist = dst
            rowidx = idx
            rowval = row

        idx = idx + 1

#     print("Row selected: ", rowidx) #Alternate_df.loc[rowidx,:]\
#     print("Min. distance: ", mindist)
#     print("Matrix shape: ", abalone_alternate_df_np.shape)
    abalone_new_df_list.append(rowval)
    abalone_alternate_df = np.delete(abalone_alternate_df_np, rowidx, 0)
    idxlist.append(rowidx)


abalone_new_df = pd.DataFrame(np.vstack(abalone_new_df_list))

print("The shape of the extracted data is: ")
print(abalone_new_df.shape)
print("-------------------------------------------------------------------")

The shape of the extracted data is: 
(60, 8)
-------------------------------------------------------------------


In [22]:
##################################################### Variance Sampling: Phase 2 ################################################

abalone_alternate_source_df = abalone_source_df.copy()
abalone_alternate_source_df_np = abalone_alternate_source_df.to_numpy()

idxlist2 = []
abalone_final_df_list = []

for row_nm in abalone_new_df_list:
    min_dist = -99
    row_idx = 0
    idx_val = 0
    for row_alt in abalone_alternate_source_df_np:
        dst = distance.euclidean(row_alt, row_nm)
        if(dst >= mindist):
            min_dist = dst
            row_idx = idx_val
            row_val = row_alt

        idx_val = idx_val + 1

#     print("Row selected: ", row_idx) #Alternate_df.loc[rowidx,:]\
#     print("Min. distance: ", min_dist)
#     print("Matrix shape: ", abalone_alternate_source_df_np.shape)
    abalone_final_df_list.append(row_val)
    abalone_alternate_source_df_np = np.delete(abalone_alternate_source_df_np, row_idx, 0)
    idxlist2.append(row_idx)


abalone_final_df = pd.DataFrame(np.vstack(abalone_final_df_list), columns= abalone_alternate_source_df.columns)

print("Shape of source before :",abalone_source_df.shape)
abalone_source_df = pd.DataFrame(np.vstack(abalone_alternate_source_df_np), columns= abalone_source_df.columns)
print("Shape of source after :", abalone_source_df.shape)

print("----------------------------------------------")
print("Shape of target before :", abalone_tgt_df.shape)
abalone_tgt_df = pd.concat([abalone_tgt_df, abalone_final_df], ignore_index=True)
print("Shape of target after :", abalone_tgt_df.shape)

print("----------------------------------------------")

Shape of source before : (2936, 8)
Shape of source after : (2876, 8)
----------------------------------------------
Shape of target before : (1241, 8)
Shape of target after : (1301, 8)
----------------------------------------------


In [23]:
################################## Importance Sampling: Manhattan Distance ######################################################

abalone_source_df["ManDis"] = ""

abalone_tgt_df_mean = []
prow = abalone_tgt_df.mean()
abalone_tgt_df_mean = [prow.Sex, prow.Length, prow.Diameter, prow.Height, prow.Shucked_weight,
       prow.Viscera_weight, prow.Shell_weight, prow.Rings]

rowidx = 0

for row in abalone_source_df.itertuples():
    row_list =[row.Sex, row.Length, row.Diameter, row.Height, row.Shucked_weight,
       row.Viscera_weight, row.Shell_weight, row.Rings]

    man_dis = 0
    for i in range(0, len(row_list)):
        tempval = abalone_tgt_df_mean[i] - row_list[i]
        man_dis = man_dis + abs(tempval)

    abalone_source_df.loc[rowidx,"ManDis"] = man_dis
    rowidx = rowidx + 1

abalone_source_df = abalone_source_df.sort_values(by =['ManDis'])
abalone_source_df = abalone_source_df.head(734) 
abalone_source_df = abalone_source_df.drop(['ManDis'], axis =1)
abalone_source_df = abalone_source_df.reset_index(drop=True)

print("Target Set: ", abalone_tgt_df.shape)
print("Source Set: ", abalone_source_df.shape)

#################### Splitting into features and target ####################
target_abalone = ['Rings']

abalone_source_df_y = abalone_source_df[target_abalone]
abalone_source_df_X = abalone_source_df.drop(target_abalone, axis = 1)

features_abalone = abalone_source_df_X.columns
print("---------------------------")

###############################################################################################################################################

Target Set:  (1301, 8)
Source Set:  (734, 8)
---------------------------


In [ ]:
#################################### STrAdaBoost.R2 Abalone ################################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

n_estimators = 100
steps = 30
fold = 10
random_state = np.random.RandomState(1)

r2scorelist_stradaboost_abalone = []
rmselist_stradaboost_abalone = []

kf = KFold(n_splits = 20)

for train_idx, test_idx in kf.split(abalone_tgt_df):
        
    abalone_train_df_X = abalone_tgt_df.iloc[train_idx].loc[:, features_abalone]
    abalone_test_df_X = abalone_tgt_df.iloc[test_idx][features_abalone]
    abalone_train_df_y = abalone_tgt_df.iloc[train_idx].loc[:,target_abalone]
    abalone_test_df_y = abalone_tgt_df.loc[test_idx][target_abalone] 
    
    abalone_X_df = pd.concat([abalone_source_df_X, abalone_train_df_X], ignore_index=True)
    abalone_y_df = pd.concat([abalone_source_df_y, abalone_train_df_y], ignore_index=True)

    abalone_np_train_X = abalone_X_df.to_numpy()
    abalone_np_train_y = abalone_y_df.to_numpy()

    abalone_np_test_X = abalone_test_df_X.to_numpy()
    abalone_np_test_y = abalone_test_df_y.to_numpy()

    abalone_np_train_y_list = abalone_np_train_y.ravel()
    abalone_np_test_y_list = abalone_np_test_y.ravel()
    
    sample_size = [len(abalone_source_df_X), len(abalone_train_df_X)]

    model_stradaboost_abalone = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth=6),
                        n_estimators = n_estimators, sample_size = sample_size,
                        steps = steps, fold = fold, random_state = random_state)


    model_stradaboost_abalone.fit(abalone_np_train_X, abalone_np_train_y_list)
    y_pred_stradaboost_abalone = model_stradaboost_abalone.predict(abalone_np_test_X)

    mse_stradaboost_abalone = sqrt(mean_squared_error(abalone_np_test_y, y_pred_stradaboost_abalone))
    rmselist_stradaboost_abalone.append(mse_stradaboost_abalone)
        
    r2_score_stradaboost_abalone = pearsonr(abalone_np_test_y_list, y_pred_stradaboost_abalone)
    r2_score_stradaboost_abalone = (r2_score_stradaboost_abalone[0])**2
    r2scorelist_stradaboost_abalone.append(r2_score_stradaboost_abalone)

print("RMSE List of STrAdaboostR2:", rmselist_stradaboost_abalone)
print("R^2 List of STrAdaboostR2:", r2scorelist_stradaboost_abalone)

print("\n")

print("Mean, STDev of RMSE:", statistics.mean(rmselist_stradaboost_abalone), statistics.stdev(rmselist_stradaboost_abalone))
print("Mean, STDev of R^2:", statistics.mean(r2scorelist_stradaboost_abalone), statistics.stdev(r2scorelist_stradaboost_abalone))



Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
RMSE List of STrAdaboostR2: [1.4998097152267589, 1.8162314223167797, 2.095896899220678, 2.782995910293012, 1.6713044531991006, 1.7085902984893293, 1.7711095331330846, 1.7775509731209713, 1.4719510937976386, 1.417697114433143, 1.831086316867

In [26]:
########################### TwoStagetrAdaBoostR2 Abalone #######################################

def get_estimator(**kwargs):
    return DecisionTreeRegressor(max_depth = 6)

kwargs_TwoTrAda = {'steps': 30,
                    'fold': 10,
                  'learning_rate': 0.1}

#####################################################################################################################

from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2

print("Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

r2scorelist_TwoTrAda_abalone = []
rmselist_TwoTrAda_abalone = []

kf = KFold(n_splits = 20)

for train_idx, test_idx in kf.split(abalone_tgt_df):
        
    abalone_train_df_X = abalone_tgt_df.iloc[train_idx].loc[:, features_abalone]
    abalone_test_df_X = abalone_tgt_df.iloc[test_idx][features_abalone]
    abalone_train_df_y = abalone_tgt_df.iloc[train_idx].loc[:,target_abalone]
    abalone_test_df_y = abalone_tgt_df.loc[test_idx][target_abalone] 
    
    abalone_X_df = pd.concat([abalone_source_df_X, abalone_train_df_X], ignore_index=True)
    abalone_y_df = pd.concat([abalone_source_df_y, abalone_train_df_y], ignore_index=True)

    abalone_np_train_X = abalone_X_df.to_numpy()
    abalone_np_train_y = abalone_y_df.to_numpy()

    abalone_np_test_X = abalone_test_df_X.to_numpy()
    abalone_np_test_y = abalone_test_df_y.to_numpy()

    abalone_np_train_y_list = abalone_np_train_y.ravel()
    abalone_np_test_y_list = abalone_np_test_y.ravel()

    src_size_abalone = len(abalone_source_df_y)
    tgt_size_abalone = len(abalone_train_df_y)
    
    src_idx_abalone = np.arange(start = 0, stop = (src_size_abalone - 1), step = 1)
    tgt_idx_abalone = np.arange(start = src_size_abalone, stop = ((src_size_abalone + tgt_size_abalone)-1), step=1)


    model_TwoTrAda_abalone = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 100) 
    model_TwoTrAda_abalone.fit(abalone_np_train_X, abalone_np_train_y_list, src_idx_abalone, tgt_idx_abalone)

    y_pred_TwoTrAda_abalone = model_TwoTrAda_abalone.predict(abalone_np_test_X)

    mse_TwoTrAda_abalone = sqrt(mean_squared_error(abalone_np_test_y, y_pred_TwoTrAda_abalone))
    rmselist_TwoTrAda_abalone.append(mse_TwoTrAda_abalone)
        
    r2_score_TwoTrAda_abalone = pearsonr(abalone_np_test_y_list, y_pred_TwoTrAda_abalone)
    r2_score_TwoTrAda_abalone = (r2_score_TwoTrAda_abalone[0])**2
    r2scorelist_TwoTrAda_abalone.append(r2_score_TwoTrAda_abalone)

print("RMSE List of TrAdaboost.R2:", rmselist_TwoTrAda_abalone)
print("R^2 List of TrAdaboost.R2:", r2scorelist_TwoTrAda_abalone)

print("\n")


print("Mean, Stdev of RMSE:", statistics.mean(rmselist_TwoTrAda_abalone), statistics.stdev(rmselist_TwoTrAda_abalone))
print("Mean, Stdev of R^2:", statistics.mean(r2scorelist_TwoTrAda_abalone), statistics.stdev(r2scorelist_TwoTrAda_abalone))


print("-------------------------------------------")

Two-Stage TrAdaboost.R2
-------------------------------------------
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
RMSE List of TrAdaboost.R2: [1.8255002950533468, 1.806742623369421, 2.082957913006075, 2.803949908402955, 1.8021205965341847, 1.816802256412984, 1.9118993845681331, 2.0809715549600303, 1.6971528242537322, 1.4279090120917652, 2.0877336149485877, 2.449807334628009, 1.377511282194191, 1.5373335

In [24]:
########################### AdaBoostR2 Transfer Learning Abalone #####################################################
from sklearn.ensemble import AdaBoostRegressor


print("Adaboost.R2 Transfer Learning")
print("-------------------------------------------")

r2scorelist_AdaTr_abalone = []
rmselist_AdaTr_abalone = []

kf = KFold(n_splits = 20)

for train_idx, test_idx in kf.split(abalone_tgt_df):
        
    abalone_train_df_X = abalone_tgt_df.iloc[train_idx].loc[:, features_abalone]
    abalone_test_df_X = abalone_tgt_df.iloc[test_idx][features_abalone]
    abalone_train_df_y = abalone_tgt_df.iloc[train_idx].loc[:,target_abalone]
    abalone_test_df_y = abalone_tgt_df.loc[test_idx][target_abalone] 
    
    abalone_X_df = pd.concat([abalone_source_df_X, abalone_train_df_X], ignore_index=True)
    abalone_y_df = pd.concat([abalone_source_df_y, abalone_train_df_y], ignore_index=True)

    abalone_np_train_X = abalone_X_df.to_numpy()
    abalone_np_train_y = abalone_y_df.to_numpy()

    abalone_np_test_X = abalone_test_df_X.to_numpy()
    abalone_np_test_y = abalone_test_df_y.to_numpy()

    abalone_np_train_y_list = abalone_np_train_y.ravel()
    abalone_np_test_y_list = abalone_np_test_y.ravel()


    model_AdaTr_abalone = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 6), learning_rate = 0.1, n_estimators = 100) 
    model_AdaTr_abalone.fit(abalone_np_train_X, abalone_np_train_y_list)

    y_pred_AdaTr_abalone = model_AdaTr_abalone.predict(abalone_np_test_X)

    mse_AdaTr_abalone = sqrt(mean_squared_error(abalone_np_test_y, y_pred_AdaTr_abalone))
    rmselist_AdaTr_abalone.append(mse_AdaTr_abalone)
        
    r2_score_AdaTr_abalone = pearsonr(abalone_np_test_y_list, y_pred_AdaTr_abalone)
    r2_score_AdaTr_abalone = (r2_score_AdaTr_abalone[0])**2
    r2scorelist_AdaTr_abalone.append(r2_score_AdaTr_abalone)

print("RMSE List of Adaboost.R2 TL:", rmselist_AdaTr_abalone)
print("R^2 List of Adaboost.R2 TL:", r2scorelist_AdaTr_abalone)

print("\n")


print("Mean, Stdev of RMSE:", statistics.mean(rmselist_AdaTr_abalone), statistics.stdev(rmselist_AdaTr_abalone))
print("Mean, Stdev of R^2:", statistics.mean(r2scorelist_AdaTr_abalone), statistics.stdev(r2scorelist_AdaTr_abalone))


print("-------------------------------------------")

Adaboost.R2 Transfer Learning
-------------------------------------------
RMSE List of Adaboost.R2 TL: [1.3277896903508537, 1.7052675173224552, 2.1916373862649565, 2.816426456231147, 1.616158165937086, 1.5387178595319144, 1.609071588350818, 1.5998206869181915, 1.3784339230800686, 1.2481550439563067, 1.7854909162526802, 2.4338357422785197, 1.3191982682408112, 1.1768617174207812, 1.451056591338254, 2.0144163774265964, 1.3159667338545962, 1.8289546508311887, 1.4109664923680096, 6.665254040110584]
R^2 List of Adaboost.R2 TL: [0.5894941123646104, 0.5465537116715246, 0.47229975845416833, 0.36907994632170604, 0.5489736408807163, 0.4592799910609312, 0.21806987563641436, 0.3911557566597031, 0.2528879619277246, 0.5065739697566836, 0.41246293827863617, 0.5130302571812415, 0.6557554821001473, 0.6396941201340589, 0.4777043283445439, 0.4808412805663861, 0.5793415684228098, 0.32754610266227324, 0.5022371852953771, 0.092338549897724]


Mean, Stdev of RMSE: 1.9216739924032908 1.1927312065574358
Mean, S

In [25]:
######################### Gradient Boosting Regression Transfer Learning Abalone #######################################
from sklearn.ensemble import GradientBoostingRegressor


print("Gradient Boosting Regression Transfer Learning")
print("-------------------------------------------")

r2scorelist_GBRTL_abalone = []
rmselist_GBRTL_abalone = []

kf = KFold(n_splits = 20)

for train_idx, test_idx in kf.split(abalone_tgt_df):
        
    abalone_train_df_X = abalone_tgt_df.iloc[train_idx].loc[:, features_abalone]
    abalone_test_df_X = abalone_tgt_df.iloc[test_idx][features_abalone]
    abalone_train_df_y = abalone_tgt_df.iloc[train_idx].loc[:,target_abalone]
    abalone_test_df_y = abalone_tgt_df.loc[test_idx][target_abalone] 
    
    abalone_X_df = pd.concat([abalone_source_df_X, abalone_train_df_X], ignore_index=True)
    abalone_y_df = pd.concat([abalone_source_df_y, abalone_train_df_y], ignore_index=True)

    abalone_np_train_X = abalone_X_df.to_numpy()
    abalone_np_train_y = abalone_y_df.to_numpy()

    abalone_np_test_X = abalone_test_df_X.to_numpy()
    abalone_np_test_y = abalone_test_df_y.to_numpy()

    abalone_np_train_y_list = abalone_np_train_y.ravel()
    abalone_np_test_y_list = abalone_np_test_y.ravel()


    model_GBRTL_abalone = GradientBoostingRegressor(learning_rate = 0.1, max_depth = 6, n_estimators = 100)
    model_GBRTL_abalone.fit(abalone_np_train_X, abalone_np_train_y_list)

    y_pred_GBRTL_abalone = model_GBRTL_abalone.predict(abalone_test_df_X) ##Using dataframe instead of the numpy matrix

    mse_GBRTL_abalone = sqrt(mean_squared_error(abalone_np_test_y, y_pred_GBRTL_abalone))
    rmselist_GBRTL_abalone.append(mse_GBRTL_abalone)
        
    r2_score_GBRTL_abalone = pearsonr(abalone_np_test_y_list, y_pred_GBRTL_abalone)
    r2_score_GBRTL_abalone = (r2_score_GBRTL_abalone[0])**2
    r2scorelist_GBRTL_abalone.append(r2_score_GBRTL_abalone)


print("RMSE List of GBRTL:", rmselist_GBRTL_abalone)
print("R^2 List of GBRTL:", r2scorelist_GBRTL_abalone)

print("\n")


print("Mean, Stdev of RMSE of GBRTL:", statistics.mean(rmselist_GBRTL_abalone), statistics.stdev(rmselist_GBRTL_abalone))
print("Mean, Stdev of R^2 of GBRTL:", statistics.mean(r2scorelist_GBRTL_abalone), statistics.stdev(r2scorelist_GBRTL_abalone))


print("-------------------------------------------")

Gradient Boosting Regression Transfer Learning
-------------------------------------------
RMSE List of GBRTL: [1.30717938884893, 1.882575290197918, 2.2603257438160513, 2.672184825516755, 1.638146271226258, 1.3550094293794106, 1.1389666930315268, 1.5744599234009782, 1.2195509192961693, 1.2171928359932076, 2.0642305667456777, 2.591431197730062, 1.286257944566433, 1.0776991887966478, 1.4837909331177048, 2.35852732383673, 1.0780309622765656, 1.6041606791455096, 1.308849719779453, 6.588709801171148]
R^2 List of GBRTL: [0.5160073023338643, 0.49571927859336423, 0.46496175468189804, 0.4214724134858678, 0.5483004083236372, 0.39536224527413283, 0.195013448144278, 0.22002103093540132, 0.23500515244188466, 0.4221536722390694, 0.30493653616038424, 0.421393242754838, 0.5544780572634954, 0.5720678579235335, 0.42775789463514, 0.334590635580131, 0.5138289107870827, 0.43331995343390134, 0.5029281100830302, 0.20747707932217943]


Mean, Stdev of RMSE of GBRTL: 1.8853639818936567 1.215936343909315
Mean, S

In [50]:
from scipy import stats

stats_StAd_rmse, pval_StAd_rmse = stats.ttest_ind(rmselist_stradaboost_abalone, rmselist_AdaTr_abalone)
print(stats_StAd_rmse, pval_StAd_rmse)

stats_StGb_rmse, pval_StGb_rmse = stats.ttest_ind(rmselist_stradaboost_abalone, rmselist_GBRTL_abalone)
print(stats_StGb_rmse, pval_StGb_rmse)

stats_StTr_rmse, pval_StTr_rmse = stats.ttest_ind(rmselist_stradaboost_abalone, rmselist_TwoTrAda_abalone)
print(stats_StTr_rmse, pval_StTr_rmse)

print("\n")

stats_StAd_r2, pval_StAd_r2 = stats.ttest_ind(r2scorelist_stradaboost_abalone, r2scorelist_AdaTr_abalone)
print(stats_StAd_r2, pval_StAd_r2)

stats_StGb_r2, pval_StGb_r2 = stats.ttest_ind(r2scorelist_stradaboost_abalone, r2scorelist_GBRTL_abalone)
print(stats_StGb_r2, pval_StGb_r2)

stats_StTr_r2, pval_StTr_r2 = stats.ttest_ind(r2scorelist_stradaboost_abalone, r2scorelist_TwoTrAda_abalone)
print(stats_StTr_r2, pval_StTr_r2)

1.4774158366799477 0.1778150997140501
1.4702303687021976 0.17970051016641017
0.180939186766893 0.8609152412759344


-0.5852548329307646 0.5745091033401768
-0.36585924469666253 0.7239555216280504
0.9064702983841878 0.3911716398994435


In [54]:
#### Dependent(paired) t-test
from scipy.stats import ttest_rel

stats_StAd_rmse, pval_StAd_rmse = stats.ttest_rel(rmselist_stradaboost_abalone, rmselist_AdaTr_abalone)
print(stats_StAd_rmse, pval_StAd_rmse)

stats_StGb_rmse, pval_StGb_rmse = stats.ttest_rel(rmselist_stradaboost_abalone, rmselist_GBRTL_abalone)
print(stats_StGb_rmse, pval_StGb_rmse)

stats_StTr_rmse, pval_StTr_rmse = stats.ttest_rel(rmselist_stradaboost_abalone, rmselist_TwoTrAda_abalone)
print(stats_StTr_rmse, pval_StTr_rmse)

print("\n")

stats_StAd_r2, pval_StAd_r2 = stats.ttest_rel(r2scorelist_stradaboost_abalone, r2scorelist_AdaTr_abalone)
print(stats_StAd_r2, pval_StAd_r2)

stats_StGb_r2, pval_StGb_r2 = stats.ttest_rel(r2scorelist_stradaboost_abalone, r2scorelist_GBRTL_abalone)
print(stats_StGb_r2, pval_StGb_r2)

stats_StTr_r2, pval_StTr_r2 = stats.ttest_rel(r2scorelist_stradaboost_abalone, r2scorelist_TwoTrAda_abalone)
print(stats_StTr_r2, pval_StTr_r2)

2.3950437583222772 0.07475725214402565
2.1242400051017136 0.10085945561990267
0.32488683877032 0.7615486615910663


-1.881842769362401 0.1329977130272984
-1.2603856460659317 0.2760438976285486
2.293616600098016 0.0835237480871644


In [27]:
################################################## Kinematics ######################################################################
#### range: [0.04 - 1.45]
#### Mid of correlation variable: theta7
#### [0, 0.6] [0.6, 0.85], [0.6, 0.85]
####################################################################################################################################
target_var_Kinematics = ['y']
colnames_Kinematics = ['theta1', 'theta2', 'theta3', 'theta4', 'theta5', 'theta6', 'theta7', 'theta8', 'y']
KinematicsData_df = pd.read_csv('UCI_regression/Kinematics/kin8nm.data', header = None,  names = colnames_Kinematics)

print("Kinematics Data")
print(KinematicsData_df.shape)

############################################### Standardization ###############################################
kinematics_cols = KinematicsData_df.columns.difference(['y'])

ss = StandardScaler()
KinematicsData_df[kinematics_cols] = ss.fit_transform(KinematicsData_df[kinematics_cols])


########## Corr Kinematics ################
# print("The correlation matrix is: ")
# print(KinematicsData_df.corr().sort_values('y')['y'])

##################### Splitting in 3 equal parts #######################################
# print(KinematicsData_df.sort_values('theta7')['theta7'])

drop_col_kinematics = ['theta7']

kinematics_tgt_df = KinematicsData_df.loc[(KinematicsData_df['theta7'] <= -0.5)]
kinematics_tgt_df = kinematics_tgt_df.drop(drop_col_kinematics, axis = 1)
kinematics_tgt_df = kinematics_tgt_df.reset_index(drop=True)
print("Target Set: ",kinematics_tgt_df.shape)


kinematics_source1_df = KinematicsData_df.loc[(KinematicsData_df['theta7'] > -0.5) & (KinematicsData_df['theta7'] <= 0.5)]
kinematics_source1_df = kinematics_source1_df.drop(drop_col_kinematics, axis = 1)
kinematics_source1_df = kinematics_source1_df.reset_index(drop=True)
print("Source Set 1: ",kinematics_source1_df.shape)


kinematics_source2_df = KinematicsData_df.loc[(KinematicsData_df['theta7'] > 0.5)]
kinematics_source2_df = kinematics_source2_df.drop(drop_col_kinematics, axis = 1)
kinematics_source2_df = kinematics_source2_df.reset_index(drop=True)
print("Source Set 2: ",kinematics_source2_df.shape)


################################# Standardization #################################
from sklearn.preprocessing import StandardScaler
kinematics_cols = kinematics_tgt_df.columns.difference(['y'])

ss = StandardScaler()
kinematics_tgt_df[kinematics_cols] = ss.fit_transform(kinematics_tgt_df[kinematics_cols])
kinematics_source1_df[kinematics_cols] = ss.fit_transform(kinematics_source1_df[kinematics_cols])
kinematics_source2_df[kinematics_cols] = ss.fit_transform(kinematics_source2_df[kinematics_cols])


################################# Concatenating the source datasets #################################
kinematics_source_df = pd.concat([kinematics_source1_df, kinematics_source2_df], ignore_index = True)
kinematics_source_df = kinematics_source_df.reset_index(drop = True)
print("Final Source Set: ",kinematics_source_df.shape)


# #################### Splitting into features and target (Hide in STrAdaBoost.R2)####################
# target_kinematics = ['y']

# kinematics_source_df_y = kinematics_source_df[target_kinematics]
# kinematics_source_df_X = kinematics_source_df.drop(target_kinematics, axis = 1)

# features_kinematics = kinematics_source_df_X.columns
# print("---------------------------")
# ##############################################################################################################################################

Kinematics Data
(8192, 9)
Target Set:  (2878, 8)
Source Set 1:  (2396, 8)
Source Set 2:  (2918, 8)
Final Source Set:  (5314, 8)


In [28]:
######################################## Variance Sampling: Phase 1 ###################################################
kmeans = KMeans(n_clusters = 140, random_state=0).fit(kinematics_tgt_df)

kinematics_alternate_df = kinematics_tgt_df.copy()
kinematics_alternate_df_np = kinematics_tgt_df.to_numpy()

idxlist = []
kinematics_new_df_list = []

for rowkm in kmeans.cluster_centers_:
    mindist = -99
    rowidx = 0
    idx = 0
    for row in kinematics_alternate_df_np:
        dst = distance.euclidean(row, rowkm)

        if(dst >= mindist):
            mindist = dst
            rowidx = idx
            rowval = row

        idx = idx + 1

#     print("Row selected: ", rowidx) #Alternate_df.loc[rowidx,:]\
#     print("Min. distance: ", mindist)
#     print("Matrix shape: ", kinematics_alternate_df_np.shape)
    kinematics_new_df_list.append(rowval)
    kinematics_alternate_df = np.delete(kinematics_alternate_df_np, rowidx, 0)
    idxlist.append(rowidx)


kinematics_new_df = pd.DataFrame(np.vstack(kinematics_new_df_list))

print("Shape of dataset extracted: ")
print(kinematics_new_df.shape)
print("----------------------------------------------")

Shape of dataset extracted: 
(140, 8)
----------------------------------------------


In [29]:
##################################################### Variance Sampling: Phase 2 ################################################

kinematics_alternate_source_df = kinematics_source_df[1:].copy()
kinematics_alternate_source_df_np = kinematics_alternate_source_df.to_numpy()

idxlist2 = []
kinematics_final_df_list = []

for row_nm in kinematics_new_df_list:
    min_dist = -99
    row_idx = 0
    idx_val = 0
    for row_alt in kinematics_alternate_source_df_np:
        dst = distance.euclidean(row_alt, row_nm)
        if(dst >= mindist):
            min_dist = dst
            row_idx = idx_val
            row_val = row_alt

        idx_val = idx_val + 1

#     print("Row selected: ", row_idx) #Alternate_df.loc[rowidx,:]\
#     print("Min. distance: ", min_dist)
#     print("Matrix shape: ", Elevators_alternate_source_df_np.shape)
    kinematics_final_df_list.append(row_val)
    kinematics_alternate_source_df_np = np.delete(kinematics_alternate_source_df_np, row_idx, 0)
    idxlist2.append(row_idx)


kinematics_final_df = pd.DataFrame(np.vstack(kinematics_final_df_list), columns= kinematics_source_df.columns)

print("----------------------------------------------")
print("Shape of source before :",kinematics_source_df.shape)
kinematics_source_df = pd.DataFrame(np.vstack(kinematics_alternate_source_df_np), columns= kinematics_source_df.columns)
print("Shape of source after :", kinematics_source_df.shape)

print("----------------------------------------------")
print("Shape of target before :", kinematics_tgt_df.shape)
kinematics_tgt_df = pd.concat([kinematics_tgt_df, kinematics_final_df], ignore_index=True)
print("Shape of target after :", kinematics_tgt_df.shape)

print("----------------------------------------------")

----------------------------------------------
Shape of source before : (5314, 8)
Shape of source after : (5173, 8)
----------------------------------------------
Shape of target before : (2878, 8)
Shape of target after : (3018, 8)
----------------------------------------------


In [30]:
################################## Importance Sampling: Manhattan Distance ######################################################

kinematics_source_df["ManDis"] = ""

kinematics_tgt_df_mean = []
prow = kinematics_tgt_df.mean()
kinematics_tgt_df_mean = [prow.theta1, prow.theta2, prow.theta3, prow.theta4, prow.theta5, prow.theta6, prow.theta8, prow.y]

rowidx = 0

for row in kinematics_source_df.itertuples():
    row_list =[row.theta1, row.theta2, row.theta3, row.theta4, row.theta5, row.theta6, row.theta8, row.y]

    man_dis = 0
    for i in range(0, len(row_list)):
        tempval = kinematics_tgt_df_mean[i] - row_list[i]
        man_dis = man_dis + abs(tempval)

    kinematics_source_df.loc[rowidx,"ManDis"] = man_dis
    rowidx = rowidx + 1

kinematics_source_df = kinematics_source_df.sort_values(by =['ManDis'])
kinematics_source_df = kinematics_source_df.head(1330) 
kinematics_source_df = kinematics_source_df.drop(['ManDis'], axis =1)
kinematics_source_df = kinematics_source_df.reset_index(drop=True)

print("Target Set: ", kinematics_tgt_df.shape)
print("Source Set: ", kinematics_source_df.shape)

#################### Splitting into features and target ####################
target_kinematics = ['y']

kinematics_source_df_y = kinematics_source_df[target_kinematics]
kinematics_source_df_X = kinematics_source_df.drop(target_kinematics, axis = 1)

features_kinematics = kinematics_source_df_X.columns
print("---------------------------")
###############################################################################################################################################

Target Set:  (3018, 8)
Source Set:  (1330, 8)
---------------------------


In [12]:
#################################### STrAdaBoost.R2 Active Learning Kinematics ################################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

n_estimators = 100
steps = 30
fold = 10
random_state = np.random.RandomState(1)

r2scorelist_stradaboost_kinematics = []
rmselist_stradaboost_kinematics = []

kf = KFold(n_splits = 20)

for train_idx, test_idx in kf.split(kinematics_tgt_df):
        
    kinematics_train_df_X = kinematics_tgt_df.iloc[train_idx].loc[:, features_kinematics]
    kinematics_test_df_X = kinematics_tgt_df.iloc[test_idx][features_kinematics]
    kinematics_train_df_y = kinematics_tgt_df.iloc[train_idx].loc[:,target_kinematics]
    kinematics_test_df_y = kinematics_tgt_df.loc[test_idx][target_kinematics] 
    
    kinematics_X_df = pd.concat([kinematics_source_df_X, kinematics_train_df_X], ignore_index=True)
    kinematics_y_df = pd.concat([kinematics_source_df_y, kinematics_train_df_y], ignore_index=True)

    kinematics_np_train_X = kinematics_X_df.to_numpy()
    kinematics_np_train_y = kinematics_y_df.to_numpy()

    kinematics_np_test_X = kinematics_test_df_X.to_numpy()
    kinematics_np_test_y = kinematics_test_df_y.to_numpy()

    kinematics_np_train_y_list = kinematics_np_train_y.ravel()
    kinematics_np_test_y_list = kinematics_np_test_y.ravel()
    
    
    sample_size = [len(kinematics_source_df_X), len(kinematics_train_df_X)]

    model_stradaboost_kinematics = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth=6),
                        n_estimators = n_estimators, sample_size = sample_size,
                        steps = steps, fold = fold, random_state = random_state)


    model_stradaboost_kinematics.fit(kinematics_np_train_X, kinematics_np_train_y_list)
    y_pred_stradaboost_kinematics = model_stradaboost_kinematics.predict(kinematics_np_test_X)


    mse_stradaboost_kinematics = sqrt(mean_squared_error(kinematics_np_test_y, y_pred_stradaboost_kinematics))

    rmselist_stradaboost_kinematics.append(mse_stradaboost_kinematics)
        
    r2_score_stradaboost_kinematics = pearsonr(kinematics_np_test_y_list, y_pred_stradaboost_kinematics)
    r2_score_stradaboost_kinematics = (r2_score_stradaboost_kinematics[0])**2

    r2scorelist_stradaboost_kinematics.append(r2_score_stradaboost_kinematics)

    
print("RMSE List of STrAdaboostR2:", statistics.mean(rmselist_stradaboost_kinematics))
print("R^2 List of STrAdaboostR2:", statistics.mean(r2scorelist_stradaboost_kinematics))

print("\n")


print("Mean, STDev of RMSE:", statistics.mean(rmselist_stradaboost_kinematics), statistics.stdev(rmselist_stradaboost_kinematics))
print("Mean, STDev of R^2:", statistics.mean(r2scorelist_stradaboost_kinematics), statistics.stdev(r2scorelist_stradaboost_kinematics))


print("-------------------------------------------")

Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
RMSE List of STrAdaboostR2: 0.18341542484176596
R^2 List of STrAdaboostR2: 0.518757608436131


Mean, STDev of RMSE: 0.18341542484176596 0.017326708837706532
Mean, STDev of R^2: 0.518757608436131 0.08719351766043475
-------------------------

In [33]:
########################### TwoStagetrAdaBoostR2 Kinematics #######################################

############# Transfer Learning specifications #########################


def get_estimator(**kwargs):
    return DecisionTreeRegressor(max_depth = 6)

kwargs_TwoTrAda = {'steps': 30,
                    'fold': 10,
                  'learning_rate': 0.1}

from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2

print("Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

r2scorelist_TwoTrAda_kinematics = []
rmselist_TwoTrAda_kinematics = []

kf = KFold(n_splits = 20)

for train_idx, test_idx in kf.split(kinematics_tgt_df):
        
    kinematics_train_df_X = kinematics_tgt_df.iloc[train_idx].loc[:, features_kinematics]
    kinematics_test_df_X = kinematics_tgt_df.iloc[test_idx][features_kinematics]
    kinematics_train_df_y = kinematics_tgt_df.iloc[train_idx].loc[:,target_kinematics]
    kinematics_test_df_y = kinematics_tgt_df.loc[test_idx][target_kinematics] 
    
    kinematics_X_df = pd.concat([kinematics_source_df_X, kinematics_train_df_X], ignore_index=True)
    kinematics_y_df = pd.concat([kinematics_source_df_y, kinematics_train_df_y], ignore_index=True)

    kinematics_np_train_X = kinematics_X_df.to_numpy()
    kinematics_np_train_y = kinematics_y_df.to_numpy()

    kinematics_np_test_X = kinematics_test_df_X.to_numpy()
    kinematics_np_test_y = kinematics_test_df_y.to_numpy()

    kinematics_np_train_y_list = kinematics_np_train_y.ravel()
    kinematics_np_test_y_list = kinematics_np_test_y.ravel()

    src_size_kinematics = len(kinematics_source_df_y)
    tgt_size_kinematics = len(kinematics_train_df_y)

    src_idx_kinematics = np.arange(start = 0, stop = (src_size_kinematics - 1), step = 1)
    tgt_idx_kinematics = np.arange(start = src_size_kinematics, stop = ((src_size_kinematics + tgt_size_kinematics)-1), step = 1)


    model_TwoTrAda_kinematics = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 100)
    model_TwoTrAda_kinematics.fit(kinematics_np_train_X, kinematics_np_train_y_list, src_idx_kinematics, tgt_idx_kinematics)

    y_pred_TwoTrAda_kinematics = model_TwoTrAda_kinematics.predict(kinematics_np_test_X)

    mse_TwoTrAda_kinematics = sqrt(mean_squared_error(kinematics_np_test_y, y_pred_TwoTrAda_kinematics))
    rmselist_TwoTrAda_kinematics.append(mse_TwoTrAda_kinematics)
        
    r2_score_TwoTrAda_kinematics = pearsonr(kinematics_np_test_y_list, y_pred_TwoTrAda_kinematics)
    r2_score_TwoTrAda_kinematics = (r2_score_TwoTrAda_kinematics[0])**2
    r2scorelist_TwoTrAda_kinematics.append(r2_score_TwoTrAda_kinematics)

print("RMSE List of TrAdaboost.R2:", rmselist_TwoTrAda_kinematics)
print("R^2 List of TrAdaboostR2:", r2scorelist_TwoTrAda_kinematics)

print("\n")


print("Mean, STDev of RMSE:", statistics.mean(rmselist_TwoTrAda_kinematics), statistics.stdev(rmselist_TwoTrAda_kinematics))
print("Mean, STDev of R^2:", statistics.mean(r2scorelist_TwoTrAda_kinematics), statistics.stdev(r2scorelist_TwoTrAda_kinematics))


print("-------------------------------------------")

Two-Stage TrAdaboost.R2
-------------------------------------------
RMSE List of TrAdaboost.R2: [0.17542033541840035, 0.17113962752390255, 0.1951171329199497, 0.18362717608325682, 0.19125173354375996, 0.1844194469064759, 0.1757610213877466, 0.18594419690577432, 0.17402287481281487, 0.15953320049867406, 0.17297012586745028, 0.1768575189940172, 0.18466759573325273, 0.18107588410326061, 0.17542612771884572, 0.16539315361147516, 0.20839389840405628, 0.18874971190954207, 0.18205135917758208, 0.2633087584159945]
R^2 List of TrAdaboostR2: [0.5281601879858504, 0.6306075074807354, 0.47940911764834176, 0.47694880204281986, 0.5086962856113789, 0.4858431457880329, 0.5133591195179663, 0.45670530257775066, 0.44836448271394314, 0.6238783054567761, 0.4842979273125778, 0.4549731816403663, 0.37563474809818453, 0.45230840128153144, 0.4753292836324454, 0.5601214144353023, 0.40978624588283724, 0.4762823295794607, 0.4513235403360286, 0.17692929832016258]


Mean, STDev of RMSE: 0.1847565439968116 0.021381949

In [31]:
########################### AdaBoostR2 Transfer Learning Kinematics #####################################################
from sklearn.ensemble import AdaBoostRegressor


print("Adaboost.R2 Transfer Learning")
print("-------------------------------------------")

r2scorelist_AdaTL_kinematics = []
rmselist_AdaTL_kinematics = []

kf = KFold(n_splits = 20)

for train_idx, test_idx in kf.split(kinematics_tgt_df):
        
    kinematics_train_df_X = kinematics_tgt_df.iloc[train_idx].loc[:, features_kinematics]
    kinematics_test_df_X = kinematics_tgt_df.iloc[test_idx][features_kinematics]
    kinematics_train_df_y = kinematics_tgt_df.iloc[train_idx].loc[:,target_kinematics]
    kinematics_test_df_y = kinematics_tgt_df.loc[test_idx][target_kinematics] 
    
    kinematics_X_df = pd.concat([kinematics_source_df_X, kinematics_train_df_X], ignore_index=True)
    kinematics_y_df = pd.concat([kinematics_source_df_y, kinematics_train_df_y], ignore_index=True)

    kinematics_np_train_X = kinematics_X_df.to_numpy()
    kinematics_np_train_y = kinematics_y_df.to_numpy()

    kinematics_np_test_X = kinematics_test_df_X.to_numpy()
    kinematics_np_test_y = kinematics_test_df_y.to_numpy()

    kinematics_np_train_y_list = kinematics_np_train_y.ravel()
    kinematics_np_test_y_list = kinematics_np_test_y.ravel()

    model_AdaTL_kinematics = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 6), learning_rate = 0.1, n_estimators = 100) 
    model_AdaTL_kinematics.fit(kinematics_np_train_X, kinematics_np_train_y_list)

    y_pred_AdaTL_kinematics = model_AdaTL_kinematics.predict(kinematics_np_test_X)

    mse_AdaTL_kinematics = sqrt(mean_squared_error(kinematics_np_test_y, y_pred_AdaTL_kinematics))
    rmselist_AdaTL_kinematics.append(mse_AdaTL_kinematics)
        
    r2_score_AdaTL_kinematics = pearsonr(kinematics_np_test_y_list, y_pred_AdaTL_kinematics)
    r2_score_AdaTL_kinematics = (r2_score_AdaTL_kinematics[0])**2
    r2scorelist_AdaTL_kinematics.append(r2_score_AdaTL_kinematics)

print("RMSE List of Adaboost.R2:", rmselist_AdaTL_kinematics)
print("R^2 List of Adaboost.R2:", r2scorelist_AdaTL_kinematics)

print("\n")


print("Mean, STDev of RMSE:", statistics.mean(rmselist_AdaTL_kinematics), statistics.stdev(rmselist_AdaTL_kinematics))
print("Mean, STDev of R^2:", statistics.mean(r2scorelist_AdaTL_kinematics), statistics.stdev(r2scorelist_AdaTL_kinematics))


print("-------------------------------------------")

Adaboost.R2 Transfer Learning
-------------------------------------------
RMSE List of Adaboost.R2: [0.176295345016129, 0.17358979306339323, 0.18356923026299643, 0.1795546558991162, 0.18797817885032578, 0.17463163937735726, 0.17137126144493392, 0.1782691440232101, 0.16427769208135556, 0.168888867425413, 0.16706388992659596, 0.1839559011433523, 0.16827703006200037, 0.18195991382183307, 0.17351957900055556, 0.16480973209558233, 0.19884382255627223, 0.17845198236471038, 0.17480098880227854, 0.2386055470573749]
R^2 List of Adaboost.R2: [0.550666248625628, 0.639112019306684, 0.5771705576297733, 0.5246965542967698, 0.5777499274897968, 0.5639777313958778, 0.5714867918591918, 0.5382907737469107, 0.5393810920128159, 0.6128368401718249, 0.5543328880141155, 0.4179958884449632, 0.4787273825419981, 0.4558891464170061, 0.4962644704223492, 0.5869026796132548, 0.4824163953249515, 0.549115883028611, 0.5307994424209936, 0.28801845496303]


Mean, STDev of RMSE: 0.17943570971373932 0.016251448024943268
Me

In [32]:
######################### Gradient Boosting Regression Transfer Learning Kinematics #######################################
from sklearn.ensemble import GradientBoostingRegressor


print("Gradient Boosting Regression Transfer Learning")
print("-------------------------------------------")

r2scorelist_GBRTL_kinematics = []
rmselist_GBRTL_kinematics = []

kf = KFold(n_splits = 20)

for train_idx, test_idx in kf.split(kinematics_tgt_df):
        
    kinematics_train_df_X = kinematics_tgt_df.iloc[train_idx].loc[:, features_kinematics]
    kinematics_test_df_X = kinematics_tgt_df.iloc[test_idx][features_kinematics]
    kinematics_train_df_y = kinematics_tgt_df.iloc[train_idx].loc[:,target_kinematics]
    kinematics_test_df_y = kinematics_tgt_df.loc[test_idx][target_kinematics] 
    
    kinematics_X_df = pd.concat([kinematics_source_df_X, kinematics_train_df_X], ignore_index=True)
    kinematics_y_df = pd.concat([kinematics_source_df_y, kinematics_train_df_y], ignore_index=True)

    kinematics_np_train_X = kinematics_X_df.to_numpy()
    kinematics_np_train_y = kinematics_y_df.to_numpy()

    kinematics_np_test_X = kinematics_test_df_X.to_numpy()
    kinematics_np_test_y = kinematics_test_df_y.to_numpy()

    kinematics_np_train_y_list = kinematics_np_train_y.ravel()
    kinematics_np_test_y_list = kinematics_np_test_y.ravel()


    model_GBRTL_kinematics = GradientBoostingRegressor(learning_rate = 0.1, max_depth = 6, n_estimators = 100)
    model_GBRTL_kinematics.fit(kinematics_np_train_X, kinematics_np_train_y_list)

    y_pred_GBRTL_kinematics = model_GBRTL_kinematics.predict(kinematics_test_df_X) ##Using dataframe instead of the numpy matrix

    mse_GBRTL_kinematics = sqrt(mean_squared_error(kinematics_np_test_y, y_pred_GBRTL_kinematics))
    rmselist_GBRTL_kinematics.append(mse_GBRTL_kinematics)
        
    r2_score_GBRTL_kinematics = pearsonr(kinematics_np_test_y_list, y_pred_GBRTL_kinematics)
    r2_score_GBRTL_kinematics = (r2_score_GBRTL_kinematics[0])**2

    r2scorelist_GBRTL_kinematics.append(r2_score_GBRTL_kinematics)


print("RMSE List of GBRTL:", rmselist_GBRTL_kinematics)
print("R^2 List of GBRTL:", r2scorelist_GBRTL_kinematics)

print("\n")

    
print("Mean, STDev of RMSE:", statistics.mean(rmselist_GBRTL_kinematics), statistics.stdev(rmselist_GBRTL_kinematics))
print("Mean, STDev of R^2:", statistics.mean(r2scorelist_GBRTL_kinematics), statistics.stdev(r2scorelist_GBRTL_kinematics))


print("-------------------------------------------")

Gradient Boosting Regression Transfer Learning
-------------------------------------------
RMSE List of GBRTL: [0.13846565258611707, 0.1332776563974172, 0.14384969460672822, 0.14267695584145648, 0.1531172531356153, 0.1542511388899997, 0.14299901051706873, 0.145230268843733, 0.13263984168751614, 0.13347869965390638, 0.14099585704703788, 0.15198912103205348, 0.1488822429904741, 0.14935628102976958, 0.14820418915256225, 0.1410216533861686, 0.15723785234486395, 0.1504925734184155, 0.138822779765777, 0.2839577465379882]
R^2 List of GBRTL: [0.7073768586224881, 0.7741033876945054, 0.725095267962796, 0.6989346726387845, 0.6874649842219676, 0.6399401415914576, 0.6738438916640964, 0.683262435503045, 0.6778742393062949, 0.7398668213016746, 0.6655507175058062, 0.5938379268071408, 0.5909848156540823, 0.6212082567761256, 0.6263216127539508, 0.6697771369598647, 0.6763116038484936, 0.6669942046995561, 0.6970386144783225, 0.11034989054657723]


Mean, STDev of RMSE: 0.15154732344323343 0.031972765481382

In [52]:
from scipy import stats

stats_StAd_rmse, pval_StAd_rmse = stats.ttest_ind(rmselist_stradaboost_kinematics, rmselist_AdaTL_kinematics)
print(stats_StAd_rmse, pval_StAd_rmse)

stats_StGb_rmse, pval_StGb_rmse = stats.ttest_ind(rmselist_stradaboost_kinematics, rmselist_GBRTL_kinematics)
print(stats_StGb_rmse, pval_StGb_rmse)

stats_StTr_rmse, pval_StTr_rmse = stats.ttest_ind(rmselist_stradaboost_kinematics, rmselist_TwoTrAda_kinematics)
print(stats_StTr_rmse, pval_StTr_rmse)

print("\n")

stats_StAd_r2, pval_StAd_r2 = stats.ttest_ind(r2scorelist_stradaboost_kinematics, r2scorelist_AdaTL_kinematics)
print(stats_StAd_r2, pval_StAd_r2)

stats_StGb_r2, pval_StGb_r2 = stats.ttest_ind(r2scorelist_stradaboost_kinematics, r2scorelist_GBRTL_kinematics)
print(stats_StGb_r2, pval_StGb_r2)

stats_StTr_r2, pval_StTr_r2 = stats.ttest_ind(r2scorelist_stradaboost_kinematics, r2scorelist_TwoTrAda_kinematics)
print(stats_StTr_r2, pval_StTr_r2)

2.239211166142098 0.05549370834515708
5.710744979214197 0.00044878897195905153
12.933901052446585 1.2086974283329565e-06


0.15020428465586927 0.8843214418707841
-0.580453091569201 0.5775889224254931
-5.759859685898271 0.0004241772800097817


In [53]:
#### Dependent(paired) t-test
from scipy.stats import ttest_rel


stats_StAd_rmse, pval_StAd_rmse = stats.ttest_rel(rmselist_stradaboost_kinematics, rmselist_AdaTL_kinematics)
print(stats_StAd_rmse, pval_StAd_rmse)

stats_StGb_rmse, pval_StGb_rmse = stats.ttest_rel(rmselist_stradaboost_kinematics, rmselist_GBRTL_kinematics)
print(stats_StGb_rmse, pval_StGb_rmse)

stats_StTr_rmse, pval_StTr_rmse = stats.ttest_rel(rmselist_stradaboost_kinematics, rmselist_TwoTrAda_kinematics)
print(stats_StTr_rmse, pval_StTr_rmse)

print("\n")

stats_StAd_r2, pval_StAd_r2 = stats.ttest_rel(r2scorelist_stradaboost_kinematics, r2scorelist_AdaTL_kinematics)
print(stats_StAd_r2, pval_StAd_r2)

stats_StGb_r2, pval_StGb_r2 = stats.ttest_rel(r2scorelist_stradaboost_kinematics, r2scorelist_GBRTL_kinematics)
print(stats_StGb_r2, pval_StGb_r2)

stats_StTr_r2, pval_StTr_r2 = stats.ttest_rel(r2scorelist_stradaboost_kinematics, r2scorelist_TwoTrAda_kinematics)
print(stats_StTr_r2, pval_StTr_r2)

3.6296148561965667 0.02216745053260249
6.519298221944031 0.0028583835971875094
13.492049030039581 0.0001746222481559807


0.7297095707949993 0.5060026819401974
-4.064041932226505 0.015297728196215526
-9.89590741109181 0.0005852307476539434


In [34]:
################################### Computer Activity ###########################################
colnames_CompAct = ['lread', 'lwrite', 'scall', 'sread', 'swrite', 'fork', 'exec', 'rchar', 'wchar', 'pgout', 'ppgout', 
                    'pgfree', 'pgscan', 'atch', 'pgin', 'ppgin', 'pflt', 'vflt', 'runqsz', 'freemem', 'freeswap', 'usr' ]
CompActData_df = pd.read_csv('UCI_regression/ComputerActivity/cpu_act.data', header = None, names = colnames_CompAct)
print("Computer Activity Data")
print(CompActData_df.shape)

##################### Corr Computer Activity #####################
# print("The correlation matrix is: ")
# print(CompActData_df.corr().sort_values('usr')['usr'])

##################### Splitting in 3 equal parts #######################################
# print(CompActData_df.sort_values('pgin')['pgin'])

drop_col_compact = ['pgin']

compact_tgt_df = CompActData_df.loc[(CompActData_df['pgin'] <= 1.0)]
compact_tgt_df = compact_tgt_df.drop(drop_col_compact, axis = 1)
compact_tgt_df = compact_tgt_df.reset_index(drop=True)
print("Target Set: ",compact_tgt_df.shape)


compact_source1_df = CompActData_df.loc[(CompActData_df['pgin'] > 1.0) & (CompActData_df['pgin'] <= 6.0)]
compact_source1_df = compact_source1_df.drop(drop_col_compact, axis = 1)
compact_source1_df = compact_source1_df.reset_index(drop = True)
print("Source Set 1: ",compact_source1_df.shape)


compact_source2_df = CompActData_df.loc[(CompActData_df['pgin'] > 6.0)]
compact_source2_df = compact_source2_df.drop(drop_col_compact, axis = 1)
compact_source2_df = compact_source2_df.reset_index(drop = True)
print("Source Set 2: ",compact_source2_df.shape)


################################# Standardization #################################
from sklearn.preprocessing import StandardScaler
compact_cols = compact_tgt_df.columns.difference(['usr'])

ss = StandardScaler()
compact_tgt_df[compact_cols] = ss.fit_transform(compact_tgt_df[compact_cols])
compact_source1_df[compact_cols] = ss.fit_transform(compact_source1_df[compact_cols])
compact_source2_df[compact_cols] = ss.fit_transform(compact_source2_df[compact_cols])


##################### Concatenating the source datasets #####################
compact_source_df = pd.concat([compact_source1_df, compact_source2_df], ignore_index = True)
compact_source_df = compact_source_df.reset_index(drop = True)
print("Final Source Set: ",compact_source_df.shape)


# #################### Splitting into features and target (Hide for STrAdaBoost.R2)####################
# target_compact = ['usr']

# compact_source_df_y = compact_source_df[target_compact]
# compact_source_df_X = compact_source_df.drop(target_compact, axis = 1)

# features_compact = compact_source_df_X.columns
# print("---------------------------")
# ##############################################################################################################################################

Computer Activity Data
(8192, 22)
Target Set:  (2766, 21)
Source Set 1:  (2539, 21)
Source Set 2:  (2887, 21)
Final Source Set:  (5426, 21)


In [35]:
######################################## Variance Sampling: Phase 1 ###################################################
kmeans = KMeans(n_clusters = 135, random_state=0).fit(compact_tgt_df)

compact_alternate_df = compact_tgt_df.copy()
compact_alternate_df_np = compact_tgt_df.to_numpy()

idxlist = []
compact_new_df_list = []

for rowkm in kmeans.cluster_centers_:
    mindist = -99
    rowidx = 0
    idx = 0
    for row in compact_alternate_df_np:
        dst = distance.euclidean(row, rowkm)

        if(dst >= mindist):
            mindist = dst
            rowidx = idx
            rowval = row

        idx = idx + 1

#     print("Row selected: ", rowidx) #Alternate_df.loc[rowidx,:]\
#     print("Min. distance: ", mindist)
#     print("Matrix shape: ", compact_alternate_df_np.shape)
    compact_new_df_list.append(rowval)
    compact_alternate_df = np.delete(compact_alternate_df_np, rowidx, 0)
    idxlist.append(rowidx)


compact_new_df = pd.DataFrame(np.vstack(compact_new_df_list))

print("The dataset extracted is: ")
print(compact_new_df.shape)
print("------------------------------------------------------")

##################################################### Variance Sampling: Phase 2 ################################################

compact_alternate_source_df = compact_source_df.copy()
compact_alternate_source_df_np = compact_alternate_source_df.to_numpy()

idxlist2 = []
compact_final_df_list = []

for row_nm in compact_new_df_list:
    min_dist = -99
    row_idx = 0
    idx_val = 0
    for row_alt in compact_alternate_source_df_np:
        dst = distance.euclidean(row_alt, row_nm)
        if(dst >= mindist):
            min_dist = dst
            row_idx = idx_val
            row_val = row_alt

        idx_val = idx_val + 1

#     print("Row selected: ", row_idx) #Alternate_df.loc[rowidx,:]\
#     print("Min. distance: ", min_dist)
#     print("Matrix shape: ", compact_alternate_source_df_np.shape)
    compact_final_df_list.append(row_val)
    compact_alternate_source_df_np = np.delete(compact_alternate_source_df_np, row_idx, 0)
    idxlist2.append(row_idx)


compact_final_df = pd.DataFrame(np.vstack(compact_final_df_list), columns= compact_alternate_source_df.columns)

print("Shape of source before :",compact_source_df.shape)
compact_source_df = pd.DataFrame(np.vstack(compact_alternate_source_df_np), columns= compact_source_df.columns)
print("Shape of source after :", compact_source_df.shape)

print("----------------------------------------------")

print("Shape of target before :", compact_tgt_df.shape)
Elevators_tgt_df = pd.concat([compact_tgt_df, compact_final_df], ignore_index=True)
print("Shape of target after :", compact_tgt_df.shape)

print("----------------------------------------------")

The dataset extracted is: 
(135, 21)
------------------------------------------------------
Shape of source before : (5426, 21)
Shape of source after : (5291, 21)
----------------------------------------------
Shape of target before : (2766, 21)
Shape of target after : (2766, 21)
----------------------------------------------


In [36]:
################################## Importance Sampling: Manhattan Distance ######################################################
compact_source_df["ManDis"] = ""

compact_tgt_df_mean = []
prow = compact_tgt_df.mean()
compact_tgt_df_mean = [prow.lread, prow.lwrite, prow.scall, prow.sread, prow.swrite, prow.fork, prow.exec, prow.rchar,
       prow.wchar, prow.pgout, prow.ppgout, prow.pgfree, prow.pgscan, prow.atch, prow.ppgin, prow.pflt, prow.vflt, prow.runqsz,
        prow.freemem, prow.freeswap, prow.usr]

rowidx = 0

for row in compact_source_df.itertuples():
    row_list =[row.lread, row.lwrite, row.scall, row.sread, row.swrite, row.fork, row.exec, row.rchar,
       row.wchar, row.pgout, row.ppgout, row.pgfree, row.pgscan, row.atch, row.ppgin, row.pflt, row.vflt, row.runqsz,
        row.freemem, row.freeswap, row.usr]

    man_dis = 0
    for i in range(0, len(row_list)):
        tempval = compact_tgt_df_mean[i] - row_list[i]
        man_dis = man_dis + abs(tempval)

    compact_source_df.loc[rowidx,"ManDis"] = man_dis
    rowidx = rowidx + 1


# kinematics_source_df = kinematics_source_df.sort_values(by =['ManDis'])
# kinematics_source_df = kinematics_source_df.head(1330) 
# kinematics_source_df = kinematics_source_df.drop(['ManDis'], axis =1)
# kinematics_source_df = kinematics_source_df.reset_index(drop=True)

# print("Target Set: ", kinematics_tgt_df.shape)
# print("Source Set: ", kinematics_source_df.shape)

compact_source_df = compact_source_df.sort_values(by =['ManDis'])
compact_source_df = compact_source_df.head(1356) #### For Computer Activity 650 instances
compact_source_df = compact_source_df.drop(['ManDis'], axis =1)
compact_source_df = compact_source_df.reset_index(drop=True)

print("Target Set: ", compact_tgt_df.shape)
print("Source Set: ", compact_source_df.shape)


#################### Splitting into features and target ####################
target_compact = ['usr']

compact_source_df_y = compact_source_df[target_compact]
compact_source_df_X = compact_source_df.drop(target_compact, axis = 1)

features_compact = compact_source_df_X.columns
print("---------------------------")
###############################################################################################################################################

Target Set:  (2766, 21)
Source Set:  (1356, 21)
---------------------------


In [20]:
#################################### STrAdaBoost.R2 compAct ################################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

n_estimators = 100
steps = 30
fold = 10
random_state = np.random.RandomState(1)

r2scorelist_stradaboost_compact = []
rmselist_stradaboost_compact = []

print("STrAdaboost.R2 (50 iter)")
print("-------------------------------------------")

kf = KFold(n_splits = 20)

for train_idx, test_idx in kf.split(compact_tgt_df):
        
    compact_train_df_X = compact_tgt_df.iloc[train_idx].loc[:, features_compact]
    compact_test_df_X = compact_tgt_df.iloc[test_idx][features_compact]
    compact_train_df_y = compact_tgt_df.iloc[train_idx].loc[:,target_compact]
    compact_test_df_y = compact_tgt_df.loc[test_idx][target_compact] 
    
    compact_X_df = pd.concat([compact_source_df_X, compact_train_df_X], ignore_index=True)
    compact_y_df = pd.concat([compact_source_df_y, compact_train_df_y], ignore_index=True)

    compact_np_train_X = compact_X_df.to_numpy()
    compact_np_train_y = compact_y_df.to_numpy()

    compact_np_test_X = compact_test_df_X.to_numpy()
    compact_np_test_y = compact_test_df_y.to_numpy()

    compact_np_train_y_list = compact_np_train_y.ravel()
    compact_np_test_y_list = compact_np_test_y.ravel()
    
    sample_size = [len(compact_source_df_X), len(compact_train_df_X)]

    model_stradaboost_compact = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth=6),
                        n_estimators = n_estimators, sample_size = sample_size,
                        steps = steps, fold = fold, random_state = random_state)


    model_stradaboost_compact.fit(compact_np_train_X, compact_np_train_y_list)
    y_pred_stradaboost_compact = model_stradaboost_compact.predict(compact_np_test_X)
    
    mse_stradaboost_compact = sqrt(mean_squared_error(compact_np_test_y, y_pred_stradaboost_compact))
    rmselist_stradaboost_compact.append(mse_stradaboost_compact)
        
    r2_score_stradaboost_compact = pearsonr(compact_np_test_y_list, y_pred_stradaboost_compact)
    r2_score_stradaboost_compact = (r2_score_stradaboost_compact[0])**2
    r2scorelist_stradaboost_compact.append(r2_score_stradaboost_compact)
        

print("RMSE List of STrAdaboost.R2:", rmselist_stradaboost_compact)
print("R^2 List of STrAdaboost.R2:", r2scorelist_stradaboost_compact)

print("\n")
        
print("Mean, STDev of RMSE:", statistics.mean(rmselist_stradaboost_compact), statistics.stdev(rmselist_stradaboost_compact))
print("Mean, STDev of R^2:", statistics.mean(r2scorelist_stradaboost_compact), statistics.stdev(r2scorelist_stradaboost_compact))



STrAdaboost.R2 (50 iter)
-------------------------------------------
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
RMSE List of STrAdaboost.R2: [2.6186617707787057, 2.6159304095258746, 2.305655221463425, 2.286693451050622, 2.4407287618657896, 2.750041727772971, 2.4052096465625996, 2.37

In [ ]:
########################### TwoStagetrAdaBoostR2 compAct #######################################

############# Transfer Learning specifications #########################

def get_estimator(**kwargs):
    return DecisionTreeRegressor(max_depth = 6)

kwargs_TwoTrAda = {'steps': 30,
                    'fold': 10,
                  'learning_rate': 0.1}

#####################################################################################################################

from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2

print("Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

r2scorelist_TwoTrAda_compact = []
rmselist_TwoTrAda_compact = []

kf = KFold(n_splits = 20)

for train_idx, test_idx in kf.split(compact_tgt_df):
        
    compact_train_df_X = compact_tgt_df.iloc[train_idx].loc[:, features_compact]
    compact_test_df_X = compact_tgt_df.iloc[test_idx][features_compact]
    compact_train_df_y = compact_tgt_df.iloc[train_idx].loc[:,target_compact]
    compact_test_df_y = compact_tgt_df.loc[test_idx][target_compact] 
    
    compact_X_df = pd.concat([compact_source_df_X, compact_train_df_X], ignore_index=True)
    compact_y_df = pd.concat([compact_source_df_y, compact_train_df_y], ignore_index=True)

    compact_np_train_X = compact_X_df.to_numpy()
    compact_np_train_y = compact_y_df.to_numpy()

    compact_np_test_X = compact_test_df_X.to_numpy()
    compact_np_test_y = compact_test_df_y.to_numpy()

    compact_np_train_y_list = compact_np_train_y.ravel()
    compact_np_test_y_list = compact_np_test_y.ravel()

    src_size_compact = len(compact_source_df_y)
    tgt_size_compact = len(compact_train_df_y)
    
    src_idx_compact = np.arange(start=0, stop=(src_size_compact - 1), step=1)
    tgt_idx_compact = np.arange(start=src_size_compact, stop=((src_size_compact + tgt_size_compact)-1), step=1)

    model_TwoTrAda_compact = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 100)
    model_TwoTrAda_compact.fit(compact_np_train_X, compact_np_train_y_list, src_idx_compact, tgt_idx_compact)

    y_pred_TwoTrAda_compact = model_TwoTrAda_compact.predict(compact_np_test_X)

    mse_TwoTrAda_compact = sqrt(mean_squared_error(compact_np_test_y, y_pred_TwoTrAda_compact))

    rmselist_TwoTrAda_compact.append(mse_TwoTrAda_compact)
        
    r2_score_TwoTrAda_compact = pearsonr(compact_np_test_y_list, y_pred_TwoTrAda_compact)
    r2_score_TwoTrAda_compact = (r2_score_TwoTrAda_compact[0])**2

    r2scorelist_TwoTrAda_compact.append(r2_score_TwoTrAda_compact)


print("RMSE List of TrAdaboost.R2:", rmselist_TwoTrAda_compact)
print("R^2 List of TrAdaboost.R2:", r2scorelist_TwoTrAda_compact)

print("\n")
        
print("Mean, STDev of RMSE:", statistics.mean(rmselist_TwoTrAda_compact), statistics.stdev(rmselist_TwoTrAda_compact))
print("Mean, STDev of R^2:", statistics.mean(r2scorelist_TwoTrAda_compact), statistics.stdev(r2scorelist_TwoTrAda_compact))


print("-------------------------------------------")

Two-Stage TrAdaboost.R2
-------------------------------------------
RMSE List of TrAdaboost.R2: [2.9702401175296216, 2.8398213086500483, 2.2719593869087538, 2.476411825762623, 2.4609045612993414, 6.198808524993722, 2.481207259683971, 2.6491835756696913, 2.1923859394275365, 2.412458822791077, 2.7148319166935644, 2.791385056208226, 2.5709765029908, 2.379737982895394, 2.613565153499139, 2.5953057146543066, 2.5209379522047266, 2.4724324178261177, 2.934709375355867, 7.059435795558402]
R^2 List of TrAdaboost.R2: [0.9525656977554645, 0.9247103911402769, 0.9485050827607413, 0.973546329424961, 0.9805278879454586, 0.8877192921793586, 0.8689798276454015, 0.9598403009193035, 0.9753502584545088, 0.9756653455545333, 0.8433195718423341, 0.935812372648972, 0.9642750872859857, 0.9452343201412735, 0.790919535733309, 0.9817541966587242, 0.9733796078184057, 0.8446134467031239, 0.9194777051693357, 0.7700136180745454]


Mean, STDev of RMSE: 2.9803349595301465 1.271830543705586
Mean, STDev of R^2: 0.92081049

In [37]:
########################### AdaBoostR2 Transfer Learning compAct #####################################################
from sklearn.ensemble import AdaBoostRegressor
from keras.callbacks import EarlyStopping

#earlystopping = EarlyStopping(monitor="mean_squared_error", patience=40, verbose=1, mode='auto')
#epochs = 100, batch_size = 5, callbacks=[earlystopping])

print("Adaboost.R2 Transfer Learning")
print("-------------------------------------------")

r2scorelist_AdaTL_compact = []
rmselist_AdaTL_compact = []

kf = KFold(n_splits = 20)

for train_idx, test_idx in kf.split(compact_tgt_df):
        
    compact_train_df_X = compact_tgt_df.iloc[train_idx].loc[:, features_compact]
    compact_test_df_X = compact_tgt_df.iloc[test_idx][features_compact]
    compact_train_df_y = compact_tgt_df.iloc[train_idx].loc[:,target_compact]
    compact_test_df_y = compact_tgt_df.loc[test_idx][target_compact] 
    
    compact_X_df = pd.concat([compact_source_df_X, compact_train_df_X], ignore_index=True)
    compact_y_df = pd.concat([compact_source_df_y, compact_train_df_y], ignore_index=True)

    compact_np_train_X = compact_X_df.to_numpy()
    compact_np_train_y = compact_y_df.to_numpy()

    compact_np_test_X = compact_test_df_X.to_numpy()
    compact_np_test_y = compact_test_df_y.to_numpy()

    compact_np_train_y_list = compact_np_train_y.ravel()
    compact_np_test_y_list = compact_np_test_y.ravel()

    model_AdaTL_compact = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 6), learning_rate = 0.1, n_estimators = 100) 
    model_AdaTL_compact.fit(compact_np_train_X, compact_np_train_y_list) 

    y_pred_AdaTL_compact = model_AdaTL_compact.predict(compact_np_test_X)

    mse_AdaTL_compact = sqrt(mean_squared_error(compact_np_test_y, y_pred_AdaTL_compact))
    rmselist_AdaTL_compact.append(mse_AdaTL_compact)
        
    r2_score_AdaTL_compact = pearsonr(compact_np_test_y_list, y_pred_AdaTL_compact)
    r2_score_AdaTL_compact = (r2_score_AdaTL_compact[0])**2
    r2scorelist_AdaTL_compact.append(r2_score_AdaTL_compact)


print("RMSE List of Adaboost.R2:", rmselist_AdaTL_compact)
print("R^2 List of Adaboost.R2:", r2scorelist_AdaTL_compact)

print("\n")
        
print("Mean, STDev of RMSE:", statistics.mean(rmselist_AdaTL_compact), statistics.stdev(rmselist_AdaTL_compact))
print("Mean, STDev of R^2:", statistics.mean(r2scorelist_AdaTL_compact), statistics.stdev(r2scorelist_AdaTL_compact))

print("-------------------------------------------")

Using TensorFlow backend.


Adaboost.R2 Transfer Learning
-------------------------------------------
RMSE List of Adaboost.R2: [2.535443690352108, 2.426298773724732, 2.2348938272799956, 2.2720307594411215, 2.1412022830792727, 2.575272873148034, 2.438645764426922, 2.350458181920487, 1.9631413159595998, 2.323233068824086, 2.502448426834783, 2.64575913260798, 2.431126067435133, 2.117698660897719, 2.48284769037193, 2.313731735204539, 2.3895243909159927, 2.395170394674901, 2.5166336748144156, 2.2256611031205282]
R^2 List of Adaboost.R2: [0.9650797568784357, 0.943249479491154, 0.9520633963182226, 0.9778367242868817, 0.986051589633486, 0.98055416317473, 0.8753235972131416, 0.9683277903340444, 0.9802606795381987, 0.9777350646181845, 0.8712027784618647, 0.9393220864917845, 0.9664640952434644, 0.957505646040744, 0.8170978330599753, 0.9856217693945841, 0.9755297403070615, 0.8545132895158487, 0.9418680942975028, 0.9808271179005743]


Mean, STDev of RMSE: 2.364061090751714 0.1693977999197432
Mean, STDev of R^2: 0.94482173460

In [38]:
######################### Gradient Boosting Regression Transfer Learning compAct #######################################
from sklearn.ensemble import GradientBoostingRegressor


print("Gradient Boosting Regression Transfer Learning")
print("-------------------------------------------")

r2scorelist_GBRTL_compact = []
rmselist_GBRTL_compact = []

kf = KFold(n_splits = 20)

for train_idx, test_idx in kf.split(compact_tgt_df):
        
    compact_train_df_X = compact_tgt_df.iloc[train_idx].loc[:, features_compact]
    compact_test_df_X = compact_tgt_df.iloc[test_idx][features_compact]
    compact_train_df_y = compact_tgt_df.iloc[train_idx].loc[:,target_compact]
    compact_test_df_y = compact_tgt_df.loc[test_idx][target_compact] 
    
    compact_X_df = pd.concat([compact_source_df_X, compact_train_df_X], ignore_index=True)
    compact_y_df = pd.concat([compact_source_df_y, compact_train_df_y], ignore_index=True)

    compact_np_train_X = compact_X_df.to_numpy()
    compact_np_train_y = compact_y_df.to_numpy()

    compact_np_test_X = compact_test_df_X.to_numpy()
    compact_np_test_y = compact_test_df_y.to_numpy()

    compact_np_train_y_list = compact_np_train_y.ravel()
    compact_np_test_y_list = compact_np_test_y.ravel()


    model_GBRTL_compact = GradientBoostingRegressor(learning_rate = 0.1, max_depth = 6, n_estimators = 100) #, subsample=0.5)
    model_GBRTL_compact.fit(compact_np_train_X, compact_np_train_y_list)

    y_pred_GBRTL_compact = model_GBRTL_compact.predict(compact_test_df_X) ##Using dataframe instead of the numpy matrix

    mse_GBRTL_compact = sqrt(mean_squared_error(compact_np_test_y, y_pred_GBRTL_compact))
    rmselist_GBRTL_compact.append(mse_GBRTL_compact)
        
    r2_score_GBRTL_compact = pearsonr(compact_np_test_y_list, y_pred_GBRTL_compact)
    r2_score_GBRTL_compact = (r2_score_GBRTL_compact[0])**2
    r2scorelist_GBRTL_compact.append(r2_score_GBRTL_compact)


print("RMSE List of GBRTL:", rmselist_GBRTL_compact)
print("R^2 List of GBRTL:", r2scorelist_GBRTL_compact)

print("\n")
        
print("Mean, STDev of RMSE:", statistics.mean(rmselist_GBRTL_compact), statistics.stdev(rmselist_GBRTL_compact))
print("Mean, STDev of R^2:", statistics.mean(r2scorelist_GBRTL_compact), statistics.stdev(r2scorelist_GBRTL_compact))

print("-------------------------------------------")

Gradient Boosting Regression Transfer Learning
-------------------------------------------
RMSE List of GBRTL: [1.8769832064074004, 1.839068187232694, 1.6892904094741985, 1.8731157550440558, 1.6923472358735283, 1.9557133699522073, 1.9244075670984566, 1.8537859165297037, 1.4986902468067873, 2.070194872117078, 1.9740079245403535, 2.0389940924389305, 1.8645251505450071, 1.8267852943887875, 1.7549144195975659, 1.7432056054850908, 1.961806199074465, 1.9179607090173094, 2.0526430319915447, 1.639691371854903]
R^2 List of GBRTL: [0.9777894148595155, 0.9637255961887524, 0.9688288503731334, 0.9838973778844392, 0.989570329182047, 0.9890166954317555, 0.9136927881624313, 0.9776499953378783, 0.9858672511240647, 0.9817186901102682, 0.9101391595719984, 0.9619646337073731, 0.9774351282228092, 0.9670961098250015, 0.8984919512130616, 0.9908156285660206, 0.9829854473392557, 0.8947072753901288, 0.9561254995560162, 0.9876661845510726]


Mean, STDev of RMSE: 1.8524065282735034 0.14784143034874064
Mean, STDev